<a href="https://colab.research.google.com/github/arielsiman-tov/multimodal-LSTM-Lyric-Generation-/blob/main/DL3_lyric_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Deep Learning- lyric_RNN_model**
#### **Tal Klein - 209234103 | Ariel Siman Tov - 209499821**

# **Imports**

In [ ]:
%%capture
!pip install pretty_midi
!pip install torch
!pip install TextBlob

In [ ]:
import os
import random
import re
import string
import csv
import pickle
import warnings
from collections import Counter
from itertools import product
from gensim.downloader import load as gensim_load
from gensim.models import KeyedVectors
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torch.distributions.one_hot_categorical import OneHotCategorical
from torch.utils.tensorboard import SummaryWriter
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import gensim.downloader as api
from gensim.models import Word2Vec
from tqdm import tqdm
import pretty_midi
import glob
from textblob import TextBlob


# **Connect to Drive**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [ ]:
# change to your path.
path_midi_files = '/home/tak/justice-tal/content/drive/MyDrive/DL3/midi_files'
train_file = '/home/tak/justice-tal/content/drive/MyDrive/DL3/lyrics_train_set.csv'
test_file = '/home/tak/justice-tal/content/drive/MyDrive/DL3/lyrics_test_set.csv'

In [ ]:
train = pd.read_csv(train_file, header=None)
train = train[[0,1,2]]
train.columns = ['Artist', 'Song Name', 'Lyrics']
train['Lyrics'] = train['Lyrics'].apply(clean_text)
train['Lyrics'] = train['Lyrics'].apply(lambda x: ' '.join(re.sub(r'[^\w\s]', '', x).split()))
train

,Artist,Song Name,Lyrics
0,elton john,candle in the wind,goodbye norma jean newline though i never knew...
1,gerry rafferty,baker street,winding your way down on baker street newline ...
2,gerry rafferty,right down the line,you know i need your love newline got that hol...
3,2 unlimited,tribal dance,come on check it out newline come on come on n...
4,2 unlimited,let the beat control your body,let the beat control your body newline let the...
...,...,...,...
595,don henley,dirty laundry,i make my living off the evening news newline ...
596,don henley,new york minute,harry got up newline dressed all in black newl...
597,bob dylan,subterranean homesick blues,in the basement newline mixing up the medicine...
598,goldfinger,mable,i met her sunday that was yesterday newline th...


In [ ]:
test = pd.read_csv(test_file, header=None )
test = test[[0,1,2]]
test.columns = ['Artist', 'Song Name', 'Lyrics']
test['Lyrics'] = test['Lyrics'].str.replace('&', 'newline')
test['Lyrics'] = test['Lyrics'].apply(lambda x: ' '.join(re.sub(r'[^\w\s]', '', x).split()))
test

,Artist,Song Name,Lyrics
0,the bangles,eternal flame,close your eyes give me your hand darling newl...
1,billy joel,honesty,if you search for tenderness newline it isnt h...
2,cardigans,lovefool,dear i fear were facing a problem newline you ...
3,aqua,barbie girl,hiya barbie newline hi ken newline do you want...
4,blink 182,all the small things,all the small things newline true care truth b...


# **Word2vec**

## Save the model

In [ ]:
word2vec = gensim_load('word2vec-google-news-300')
word2vec.save('/home/tak/justice-tal/content/drive/MyDrive/DL3/word2vec-google-news-300.gensim')

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [ ]:
# Download the text8 corpus
corpus = api.load('text8')

# Convert the corpus to the format required by Word2Vec
sentences = [list(doc) for doc in corpus]

# Train the Word2Vec model with a vector size of 300
word_model = Word2Vec(sentences=sentences, vector_size=300, window=5, min_count=1, workers=8)

# Save the trained model
word_model.save("w2v_text8")

[==================================================] 100.0% 31.6/31.6MB downloaded


In [ ]:
# Save the trained model to Google Drive
save_path = "/home/tak/justice-tal/content/drive/MyDrive/DL3/w2v_text8.model"
word_model.save(save_path)

## Load the model

### text8

In [ ]:
model_path = "/home/tak/justice-tal/content/drive/MyDrive/DL3/w2v_text8.model"
word_model = Word2Vec.load(model_path)

### google news

In [ ]:
model_path = "/home/tak/justice-tal/content/drive/MyDrive/DL3/word2vec-google-news-300.gensim"
word_model = KeyedVectors.load(model_path)

### glove

In [ ]:
# word_model = {}
# embedding = []
# idx2word = []

# with open('/home/tak/justice-tal/content/drive/MyDrive/DL3/glove.6B.300d.txt', encoding='utf-8') as f:
#     # is just a space-separated text file in the format:
#     # word vec[0] vec[1] vec[2] ...
#     for line in f:
#         values = line.split()
#         word = values[0]
#         vec = np.asarray(values[1:], dtype='float32')
#         word_model[word] = vec
#         embedding.append(vec)
#         idx2word.append(word)

# # save the word_model
# with open('/home/tak/justice-tal/content/drive/MyDrive/DL3/glove.6B.300d.pkl', 'wb') as f:
#     pickle.dump(word_model, f)

# **Function for create data**

In [ ]:
def rename_files_to_lowercase(directory_path):
    for filename in os.listdir(directory_path):
        if filename != filename.lower():
            if filename=='depeche_mode_-_enjoy_the_silence.mid':
              filename = 'depeche_mode_-_enjoy_the_silence-2.mid'

            elif filename=='eric_clapton_-_wonderful_tonight.mid':
              filename = 'eric_clapton_-_wonderful_tonight_-_live_extnd_version_@jiji@.mid'

            elif filename=='ed_sheeran_-_thinking_out_loud.mid':
              filename = 'ed_sheeran_-_thinking_out_loud_-_violin.mid'
            # Create new file name with all lowercase letters
            new_filename = filename.lower()
            # Construct full file paths
            old_file_path = os.path.join(directory_path, filename)
            new_file_path = os.path.join(directory_path, new_filename)
            # Rename the file
            os.rename(old_file_path, new_file_path)
rename_files_to_lowercase('/home/tak/justice-tal/content/drive/MyDrive/DL3/midi_files')

In [ ]:
def clean_text(song_lyrics):
    # Replace unwanted characters and patterns
    replacements = {
        '&': 'newline',
        '  ': ' ',
        '\'': '',
        '--': ' ',
    }
    for old, new in replacements.items():
        song_lyrics = song_lyrics.replace(old, new)

    # Tokenize and clean tokens
    tokens = song_lyrics.split()
    tokens = [word.translate(str.maketrans('', '', string.punctuation)) for word in tokens]  # Remove punctuation
    tokens = [word for word in tokens if word.isalpha()]  # Keep only alphabetic tokens
    tokens = [word.lower() for word in tokens if word.lower() in word_model.wv]  # Convert to lowercase and check in vocabulary

    # Join cleaned tokens back into a single string
    return ' '.join(tokens)

In [ ]:
def csv2df(df_path, is_test=False):
    df1 = pd.read_csv(df_path, header=None)
    df = df1.iloc[:, :3]
    df.columns = ['Artist', 'Name', 'Lyrics']
    #replace with the function that cleans the text
    df['Lyrics'] = df['Lyrics'].apply(clean_text)
    df['Lyrics'] = df['Lyrics'].apply(lambda x: ' '.join(re.sub(r'[^\w\s]', '', x).split()))
    # Create the 'path' column by combining 'Artist' and 'Name' with "_-_"
    if is_test == False:
      df['Path'] = df.apply(lambda row: f"/home/tak/justice-tal/content/drive/MyDrive/DL3/midi_files/{row['Artist'].replace(' ', '_')}_-_{row['Name'].replace(' ', '_')}.mid", axis=1)
      return df
    else:
      df['Path'] = df.apply(lambda row: f"/home/tak/justice-tal/content/drive/MyDrive/DL3/midi_files/{row['Artist'].replace(' ', '_')}_-{row['Name'].replace(' ', '_')}.mid", axis=1)
      return df

In [ ]:
df = csv2df('/home/tak/justice-tal/content/drive/MyDrive/DL3/lyrics_train_set.csv',is_test=False)
X_input = []
Y_target = []
words_not_in_w2v = set()
# save all the token in the lyrics
all_tokens = []
for i, row in tqdm(df.iterrows(), total=df.shape[0], desc="Processing Rows"):
    # Extract lyrics and MIDI path
    lyrics = row['Lyrics'].split(" ")
    all_tokens.extend(lyrics)
token_counts = {token: all_tokens.count(token) for token in set(all_tokens)}
tokens_to_keep = [
    token for token, count in token_counts.items()
    if count >= 10 or token in word_model.wv
]
tokens_to_keep = list(set(tokens_to_keep))  # Ensure uniqueness
reduced_vectors = np.zeros((len(tokens_to_keep), 300))  # +1 for <UNK> token
reduced_vocab = {}  # Map <UNK> token to index 0
voc_idx = {}
for idx, token in enumerate(tokens_to_keep, start=0):  # Start indexing at 1
    if token in word_model.wv:
        reduced_vectors[idx] = word_model.wv[token]
    else:
        reduced_vectors[idx] = np.random.uniform(-0.1, 0.1, 300)
    reduced_vocab[token] = idx
    voc_idx[idx] = token

/tmp/ipykernel_1783929/1791170600.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Lyrics'] = df['Lyrics'].apply(clean_text)
/tmp/ipykernel_1783929/1791170600.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Lyrics'] = df['Lyrics'].apply(lambda x: ' '.join(re.sub(r'[^\w\s]', '', x).split()))
/tmp/ipykernel_1783929/1791170600.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats

In [ ]:
def extract_features_from_melody(index_word, ticks_per_beat, midi_file):
    """
    Extracts musical features for a specific time window from a MIDI file.

    Parameters:
        index_word (int): The index representing the word position in the MIDI timeline.
        ticks_per_beat (int): The duration of one word in ticks (time per division).
        midi_file (pretty_midi.PrettyMIDI): The MIDI file to analyze.

    Returns:
        np.ndarray: A feature vector containing various musical attributes for the time window.
    """
    # Define the time window
    start_time = index_word * ticks_per_beat
    end_time = start_time + ticks_per_beat

    # Initialize feature variables
    average_velocity = 0
    average_pitch = 0
    average_programs = 0
    num_instruments = 0
    num_notes = 0
    has_drums = 0
    onset_density = 0
    note_durations = []
    programs = []
    pitch_class_histogram = np.zeros(12)  # 12 pitch classes in an octave
    key_changes = midi_file.key_signature_changes
    key = key_changes[0].key_number if key_changes else 0

    _, temp = midi_file.get_tempo_changes()
    tempo = temp[0] if len(temp) > 0 else 120
    tempo_normalized = tempo / 300

    # Iterate through instruments and their notes
    for instrument in midi_file.instruments:
        instrument_in_range = False
        for note in instrument.notes:
            if start_time <= note.start < end_time and note.end <= end_time:
                if instrument.is_drum:
                    has_drums = 1
                instrument_in_range = True
                num_notes += 1
                average_pitch += note.pitch
                average_velocity += note.velocity
                note_durations.append(note.end - note.start)
                pitch_class_histogram[note.pitch % 12] += 1
                programs.append(instrument.program)

            elif note.start >= end_time:
                break  # Stop processing notes outside the time window

        if instrument_in_range:
            num_instruments += 1

    # Calculate aggregate features
    if num_notes > 0:
        average_velocity /= num_notes
        average_pitch /= num_notes
        onset_density = num_notes / (end_time - start_time)
        average_programs /= num_notes

    if note_durations:
        note_durations = np.array(note_durations)
        mean_duration = np.mean(note_durations)
        variance_duration = np.var(note_durations)
    else:
        mean_duration = 0
        variance_duration = 0

    # Combine features into a final feature vector
    final_features = np.concatenate((
        np.array([
            tempo_normalized,
            key,
            average_programs,
            average_velocity,
            average_pitch,
            num_instruments,
            has_drums,
            onset_density,
            mean_duration,
            variance_duration,
            0,  # Placeholder for unused chroma features
            0,  # Placeholder for unused chroma features
            0   # Placeholder for unused chroma features
        ]),
        pitch_class_histogram
    ), axis=0)

    return final_features

In [ ]:
def extract_piano_roll_features(num_of_notes_per_word, piano_roll, word_idx):
    start_idx = word_idx * num_of_notes_per_word
    end_idx = start_idx + num_of_notes_per_word
    piano_roll_for_lyric = piano_roll[:, start_idx:end_idx].transpose()
    piano_roll_slice_sum = np.sum(piano_roll_for_lyric, axis=0)  # Sum each column into a single cell
    return piano_roll_slice_sum

def prepare_train(df_path):
    """
    Prepares the training dataset by extracting features from MIDI files and corresponding lyrics.

    Parameters:
        df_path (str): Path to the CSV file containing lyrics and MIDI file paths.

    Returns:
        tuple: A tuple containing:
            - X_input (list): Combined tensors of word embeddings and MIDI features.
            - Y_target (list): Target word embeddings.
    """
    # Load the dataframe
    df = csv2df(df_path,is_test=False)
    X_input = []
    Y_target = []
    words_not_in_w2v = set()
    # save all the token in the lyrics
    all_tokens = []
    for i, row in tqdm(df.iterrows(), total=df.shape[0], desc="Processing Rows"):
        # Extract lyrics and MIDI path
        lyrics = row['Lyrics'].split(" ")
        all_tokens.extend(lyrics)
    token_counts = {token: all_tokens.count(token) for token in set(all_tokens)}
    tokens_to_keep = [
        token for token, count in token_counts.items()
        if count >= 10 or token in word_model.wv
    ]
    tokens_to_keep = list(set(tokens_to_keep))  # Ensure uniqueness
    reduced_vectors = np.zeros((len(tokens_to_keep), 300))  # +1 for <UNK> token
    reduced_vocab = {}  # Map <UNK> token to index 0
    voc_idx = {}
    for idx, token in enumerate(tokens_to_keep, start=0):  # Start indexing at 1
        if token in word_model.wv:
            reduced_vectors[idx] = word_model.wv[token]
        else:
            reduced_vectors[idx] = np.random.uniform(-0.1, 0.1, 300)
        reduced_vocab[token] = idx
        voc_idx[idx] = token
    reduced_vectors = torch.tensor(reduced_vectors, dtype=torch.float32)

    for i, row in tqdm(df.iterrows(), total=df.shape[0], desc="Processing Rows"):
        # Extract lyrics and MIDI path
        lyrics = row['Lyrics'].split(" ")
        midi_path = row['Path']

        # Attempt to load MIDI data
        try:
            midi_pretty_data = pretty_midi.PrettyMIDI(midi_path)
        except Exception as e:
            print(f"Error processing {midi_path}: {e}")
            continue

        # Calculate MIDI-related features
        avg_time = midi_pretty_data.get_end_time() / len(lyrics)
        beat_intervals = [
            x - midi_pretty_data.get_beats()[i - 1]
            for i, x in enumerate(midi_pretty_data.get_beats())
        ][1:]
        beat_freq = np.mean(beat_intervals) if beat_intervals else 0
        num_instruments = len(midi_pretty_data.instruments)
        chroma = midi_pretty_data.get_chroma().mean(axis=1) if midi_pretty_data.get_chroma().size > 0 else np.zeros(12)
        midi_pretty_data.remove_invalid_notes()
        num_of_words_in_song = len(lyrics)
        piano_roll = midi_pretty_data.get_piano_roll(fs=50)
        num_of_notes_per_word = int(piano_roll.shape[1] / num_of_words_in_song)  # Num of piano roll columns per word
        # Process each word in the lyrics (excluding the last one)
        for i in range(len(lyrics) - 1):
            current_word = lyrics[i]
            next_word = lyrics[i + 1]
            # Extract MIDI features for the current word
            try:
                word_melody = extract_features_from_melody(i, avg_time, midi_pretty_data)
            except Exception as e:
                print(f"Error extracting features for word {i} in {midi_path}: {e}")
                continue
            notes_features = extract_piano_roll_features(num_of_notes_per_word, piano_roll, i)
            # Update extracted MIDI features with additional data
            word_melody[5] = num_instruments  # Update the number of instruments
            word_melody[10] = beat_freq       # Update the beat frequency
            word_melody[11:11 + 12] = chroma   # Update the chroma features
            word_melody = np.concatenate((word_melody, notes_features), axis=0)
            # normalize word_melody
            scaler = MinMaxScaler()
            word_melody = scaler.fit_transform(word_melody.reshape(-1, 1)).flatten()
            # Check if both words are in the Word2Vec model's vocabulary
            if current_word in word_model.wv and next_word in word_model.wv:
                # Convert current word embedding to a tensor
                current_word_tensor = torch.tensor(word_model.wv[current_word])
                # Convert MIDI features to a tensor
                word_melody_tensor = torch.from_numpy(word_melody).float()
                # Concatenate word embedding and MIDI features
                combined_tensor = torch.cat((current_word_tensor, word_melody_tensor), dim=0)
                # Append tensors to the training data
                X_input.append(combined_tensor)

                # Prepare one-hot target vector
                next_word_index = reduced_vocab[next_word]  # Get the index of the next word
                one_hot_target = torch.zeros(len(reduced_vocab)).to(device)  # Create a zero vector of vocab size
                one_hot_target[next_word_index] = 1.0  # Set the target index to 1
                Y_target.append(one_hot_target)


            else:
                # Add words not in Word2Vec vocabulary to a tracking set
                words_not_in_w2v.update({current_word, next_word})

    return X_input, Y_target

In [ ]:
def prepare_test(df_path):
    """
    Prepares the test dataset by extracting features from MIDI files and corresponding lyrics.

    Parameters:
        df_path (str): Path to the CSV file containing lyrics and MIDI file paths.

    Returns:
        list: A list of lists, where each inner list contains tensors of MIDI features for a song's lyrics.
    """
    # Load the test dataframe
    df = csv2df(df_path, is_test=True)
    X_input = []
    words_not_in_w2v = set()

    for i, row in tqdm(df.iterrows(), total=df.shape[0], desc="Processing Rows"):
        # Extract lyrics and MIDI path
        lyrics = row['Lyrics'].split(" ")
        midi_path = row['Path']
        print(f"Processing MIDI file: {midi_path}")

        # Attempt to load MIDI data
        try:
            midi_pretty_data = pretty_midi.PrettyMIDI(midi_path)
        except Exception as e:
            print(f"Error processing {midi_path}: {e}")
            continue

        # Calculate MIDI-related features
        avg_time = midi_pretty_data.get_end_time() / len(lyrics)
        beat_intervals = [
            x - midi_pretty_data.get_beats()[i - 1]
            for i, x in enumerate(midi_pretty_data.get_beats())
        ][1:]
        beat_freq = np.mean(beat_intervals) if beat_intervals else 0
        chroma = midi_pretty_data.get_chroma().mean(axis=1) if midi_pretty_data.get_chroma().size > 0 else np.zeros(12)
        num_instruments = len(midi_pretty_data.instruments)
        midi_pretty_data.remove_invalid_notes()

        num_of_words_in_song = len(lyrics)

        piano_roll = midi_pretty_data.get_piano_roll(fs=50)
        num_of_notes_per_word = int(piano_roll.shape[1] / num_of_words_in_song)  # Num of piano roll columns per word

        # Process each word in the lyrics (excluding the last one)
        curr_song = []
        for j in range(len(lyrics) - 1):
            try:
                # Extract MIDI features for the current word
                word_melody = extract_features_from_melody(j, avg_time, midi_pretty_data)
                notes_features = extract_piano_roll_features(num_of_notes_per_word, piano_roll, i)

                # Update MIDI features with additional data
                word_melody[5] = num_instruments  # Update the number of instruments
                word_melody[10] = beat_freq        # Update the beat frequency
                word_melody[11:11 + 12] = chroma    # Update the chroma features
                word_melody = np.concatenate((word_melody, notes_features), axis=0)


                # normalize word_melody
                scaler = MinMaxScaler()
                word_melody = scaler.fit_transform(word_melody.reshape(-1, 1)).flatten()
                # Convert the features to a tensor and append to the current song list
                curr_song.append(torch.from_numpy(word_melody).float())
            except Exception as e:
                print(f"Error extracting features for word {j} in {midi_path}: {e}")
                continue

        # Append the song's features to the input list
        X_input.append(curr_song)

    return X_input

In [ ]:
def prepare_train_2(df_path):
    """
    Prepares the training dataset by extracting features from MIDI files and corresponding lyrics.

    Parameters:
        df_path (str): Path to the CSV file containing lyrics and MIDI file paths.

    Returns:
        tuple: A tuple containing:
            - X_input (list): Combined tensors of word embeddings and MIDI features.
            - Y_target (list): Target word embeddings.
    """
    # Load the dataframe
    df = csv2df(df_path,is_test=False)
    X_input = []
    Y_target = []
    words_not_in_w2v = set()
    # save all the token in the lyrics
    all_tokens = []
    for i, row in tqdm(df.iterrows(), total=df.shape[0], desc="Processing Rows"):
        # Extract lyrics and MIDI path
        lyrics = row['Lyrics'].split(" ")
        all_tokens.extend(lyrics)
    token_counts = {token: all_tokens.count(token) for token in set(all_tokens)}
    tokens_to_keep = [
        token for token, count in token_counts.items()
        if count >= 10 or token in word_model.wv
    ]
    tokens_to_keep = list(set(tokens_to_keep))  # Ensure uniqueness
    reduced_vectors = np.zeros((len(tokens_to_keep), 300))  # +1 for <UNK> token
    reduced_vocab = {}  # Map <UNK> token to index 0
    voc_idx = {}
    for idx, token in enumerate(tokens_to_keep, start=0):  # Start indexing at 1
        if token in word_model.wv:
            reduced_vectors[idx] = word_model.wv[token]
        else:
            reduced_vectors[idx] = np.random.uniform(-0.1, 0.1, 300)
        reduced_vocab[token] = idx
        voc_idx[idx] = token
    reduced_vectors = torch.tensor(reduced_vectors, dtype=torch.float32)

    for i, row in tqdm(df.iterrows(), total=df.shape[0], desc="Processing Rows"):
        # Extract lyrics and MIDI path
        lyrics = row['Lyrics'].split(" ")
        midi_path = row['Path']

        # Attempt to load MIDI data
        try:
            midi_pretty_data = pretty_midi.PrettyMIDI(midi_path)
        except Exception as e:
            print(f"Error processing {midi_path}: {e}")
            continue

        # Calculate MIDI-related features
        avg_time = midi_pretty_data.get_end_time() / len(lyrics)
        beat_intervals = [
            x - midi_pretty_data.get_beats()[i - 1]
            for i, x in enumerate(midi_pretty_data.get_beats())
        ][1:]
        beat_freq = np.mean(beat_intervals) if beat_intervals else 0
        num_instruments = len(midi_pretty_data.instruments)
        chroma = midi_pretty_data.get_chroma().mean(axis=1) if midi_pretty_data.get_chroma().size > 0 else np.zeros(12)
        midi_pretty_data.remove_invalid_notes()
        # Process each word in the lyrics (excluding the last one)
        for i in range(len(lyrics) - 1):
            current_word = lyrics[i]
            next_word = lyrics[i + 1]
            # Extract MIDI features for the current word
            try:
                word_melody = extract_features_from_melody(i, avg_time, midi_pretty_data)
            except Exception as e:
                print(f"Error extracting features for word {i} in {midi_path}: {e}")
                continue
            # Update extracted MIDI features with additional data
            word_melody[5] = num_instruments  # Update the number of instruments
            word_melody[10] = beat_freq       # Update the beat frequency
            word_melody[11:11 + 12] = chroma   # Update the chroma features
            # normalize word_melody
            scaler = MinMaxScaler()
            word_melody = scaler.fit_transform(word_melody.reshape(-1, 1)).flatten()
            # Check if both words are in the Word2Vec model's vocabulary
            if current_word in word_model.wv and next_word in word_model.wv:
                # Convert current word embedding to a tensor
                current_word_tensor = torch.tensor(word_model.wv[current_word])
                # Convert MIDI features to a tensor
                word_melody_tensor = torch.from_numpy(word_melody).float()
                # Concatenate word embedding and MIDI features
                combined_tensor = torch.cat((current_word_tensor, word_melody_tensor), dim=0)
                # Append tensors to the training data
                X_input.append(combined_tensor)

                # Prepare one-hot target vector
                next_word_index = reduced_vocab[next_word]  # Get the index of the next word
                one_hot_target = torch.zeros(len(reduced_vocab)).to(device)  # Create a zero vector of vocab size
                one_hot_target[next_word_index] = 1.0  # Set the target index to 1
                Y_target.append(one_hot_target)


            else:
                # Add words not in Word2Vec vocabulary to a tracking set
                words_not_in_w2v.update({current_word, next_word})

    return X_input, Y_target

In [ ]:
def prepare_test_2(df_path):
    """
    Prepares the test dataset by extracting features from MIDI files and corresponding lyrics.

    Parameters:
        df_path (str): Path to the CSV file containing lyrics and MIDI file paths.

    Returns:
        list: A list of lists, where each inner list contains tensors of MIDI features for a song's lyrics.
    """
    # Load the test dataframe
    df = csv2df(df_path, is_test=True)
    X_input = []
    words_not_in_w2v = set()

    for i, row in tqdm(df.iterrows(), total=df.shape[0], desc="Processing Rows"):
        # Extract lyrics and MIDI path
        lyrics = row['Lyrics'].split(" ")
        midi_path = row['Path']
        print(f"Processing MIDI file: {midi_path}")

        # Attempt to load MIDI data
        try:
            midi_pretty_data = pretty_midi.PrettyMIDI(midi_path)
        except Exception as e:
            print(f"Error processing {midi_path}: {e}")
            continue

        # Calculate MIDI-related features
        avg_time = midi_pretty_data.get_end_time() / len(lyrics)
        beat_intervals = [
            x - midi_pretty_data.get_beats()[i - 1]
            for i, x in enumerate(midi_pretty_data.get_beats())
        ][1:]
        beat_freq = np.mean(beat_intervals) if beat_intervals else 0
        chroma = midi_pretty_data.get_chroma().mean(axis=1) if midi_pretty_data.get_chroma().size > 0 else np.zeros(12)
        num_instruments = len(midi_pretty_data.instruments)
        midi_pretty_data.remove_invalid_notes()

        # Process each word in the lyrics (excluding the last one)
        curr_song = []
        for j in range(len(lyrics) - 1):
            try:
                # Extract MIDI features for the current word
                word_melody = extract_features_from_melody(j, avg_time, midi_pretty_data)

                # Update MIDI features with additional data
                word_melody[5] = num_instruments  # Update the number of instruments
                word_melody[10] = beat_freq        # Update the beat frequency
                word_melody[11:11 + 12] = chroma    # Update the chroma features


                # normalize word_melody
                scaler = MinMaxScaler()
                word_melody = scaler.fit_transform(word_melody.reshape(-1, 1)).flatten()
                # Convert the features to a tensor and append to the current song list
                curr_song.append(torch.from_numpy(word_melody).float())
            except Exception as e:
                print(f"Error extracting features for word {j} in {midi_path}: {e}")
                continue

        # Append the song's features to the input list
        X_input.append(curr_song)

    return X_input

# **Create and save data**

## First Features

In [ ]:
X,Y = prepare_train_2('/home/tak/justice-tal/content/drive/MyDrive/DL3/lyrics_train_set.csv')

/tmp/ipykernel_716480/1791170600.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Lyrics'] = df['Lyrics'].apply(clean_text)
/tmp/ipykernel_716480/1791170600.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Lyrics'] = df['Lyrics'].apply(lambda x: ' '.join(re.sub(r'[^\w\s]', '', x).split()))
/tmp/ipykernel_716480/1791170600.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in

KeyboardInterrupt: 

In [ ]:
X_test = prepare_test_2('/home/tak/justice-tal/content/drive/MyDrive/DL3/lyrics_test_set.csv')

Processing Rows:  20%|██        | 1/5 [00:00<00:00,  6.27it/s]

Processing MIDI file: /home/tak/justice-tal/content/drive/MyDrive/DL3/midi_files/the_bangles_-_eternal_flame.mid
Processing MIDI file: /home/tak/justice-tal/content/drive/MyDrive/DL3/midi_files/billy_joel_-_honesty.mid


Processing Rows:  40%|████      | 2/5 [00:00<00:00,  5.08it/s]

Processing MIDI file: /home/tak/justice-tal/content/drive/MyDrive/DL3/midi_files/cardigans_-_lovefool.mid


Processing Rows:  60%|██████    | 3/5 [00:00<00:00,  3.55it/s]

Processing MIDI file: /home/tak/justice-tal/content/drive/MyDrive/DL3/midi_files/aqua_-_barbie_girl.mid


Processing Rows:  80%|████████  | 4/5 [00:01<00:00,  2.26it/s]

Processing MIDI file: /home/tak/justice-tal/content/drive/MyDrive/DL3/midi_files/blink_182_-_all_the_small_things.mid


Processing Rows: 100%|██████████| 5/5 [00:01<00:00,  2.66it/s]


In [ ]:
def save_to_pickle(data, filename):
    with open(filename, 'wb') as f:
        pickle.dump(data, f)

In [ ]:
save_to_pickle((X, Y), '/home/tak/justice-tal/content/drive/MyDrive/DL3/train_data_last_new_2.pkl')

In [ ]:
save_to_pickle((X_test), '/home/tak/justice-tal/content/drive/MyDrive/DL3/test_data_last_new_2.pkl')

## Second Features

In [ ]:
X,Y = prepare_train('/home/tak/justice-tal/content/drive/MyDrive/DL3/lyrics_train_set.csv')

/tmp/ipykernel_1783929/1791170600.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Lyrics'] = df['Lyrics'].apply(clean_text)
/tmp/ipykernel_1783929/1791170600.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Lyrics'] = df['Lyrics'].apply(lambda x: ' '.join(re.sub(r'[^\w\s]', '', x).split()))
/tmp/ipykernel_1783929/1791170600.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats

Error processing /home/tak/justice-tal/content/drive/MyDrive/DL3/midi_files/beastie_boys_-_girls.mid: Could not decode key with 1 flats and mode 255


Processing Rows:  19%|█▉        | 116/600 [01:30<09:04,  1.13s/it]

Error processing /home/tak/justice-tal/content/drive/MyDrive/DL3/midi_files/eric_clapton_-_wonderful_tonight.mid: [Errno 2] No such file or directory: '/home/tak/justice-tal/content/drive/MyDrive/DL3/midi_files/eric_clapton_-_wonderful_tonight.mid'


Processing Rows:  20%|██        | 121/600 [01:36<08:59,  1.13s/it]/home/tak/.conda/envs/tal_env/lib/python3.12/site-packages/pretty_midi/pretty_midi.py:100: RuntimeWarning: Tempo, Key or Time signature change events found on non-zero tracks.  This is not a valid type 0 or type 1 MIDI file.  Tempo, Key or Time Signature may be wrong.
  warnings.warn(
Processing Rows:  21%|██        | 125/600 [01:38<05:43,  1.38it/s]/home/tak/.conda/envs/tal_env/lib/python3.12/site-packages/pretty_midi/pretty_midi.py:100: RuntimeWarning: Tempo, Key or Time signature change events found on non-zero tracks.  This is not a valid type 0 or type 1 MIDI file.  Tempo, Key or Time Signature may be wrong.
  warnings.warn(
Processing Rows:  22%|██▏       | 132/600 [01:46<08:33,  1.10s/it]

...

Processing Rows:  72%|███████▏  | 432/600 [05:11<01:33,  1.80it/s]/home/tak/.conda/envs/tal_env/lib/python3.12/site-packages/pretty_midi/pretty_midi.py:100: RuntimeWarning: Tempo, Key or Time signature change events found on non-zero tracks.  This is not a valid type 0 or type 1 MIDI file.  Tempo, Key or Time Signature may be wrong.
  warnings.warn(
Processing Rows:  78%|███████▊  | 467/600 [06:17<01:52,  1.18it/s]/home/tak/.conda/envs/tal_env/lib/python3.12/site-packages/pretty_midi/pretty_midi.py:100: RuntimeWarning: Tempo, Key or Time signature change events found on non-zero tracks.  This is not a valid type 0 or type 1 MIDI file.  Tempo, Key or Time Signature may be wrong.
  warnings.warn(
Processing Rows:  80%|███████▉  | 478/600 [06:26<01:43,  1.18it/s]/home/tak/.conda/envs/tal_env/lib/python3.12/site-packages/pretty_midi/pretty_midi.py:100: RuntimeWarning: Tempo, Key or Time signature change events found on non-zero tracks.  This is not a valid type 0 or type 1 MIDI file.  Temp

Error processing /home/tak/justice-tal/content/drive/MyDrive/DL3/midi_files/brian_mcknight_-_on_the_down_low.mid: 


Processing Rows:  85%|████████▌ | 512/600 [06:48<01:41,  1.16s/it]/home/tak/.conda/envs/tal_env/lib/python3.12/site-packages/pretty_midi/pretty_midi.py:100: RuntimeWarning: Tempo, Key or Time signature change events found on non-zero tracks.  This is not a valid type 0 or type 1 MIDI file.  Tempo, Key or Time Signature may be wrong.
  warnings.warn(
Processing Rows:  89%|████████▉ | 535/600 [07:09<00:57,  1.14it/s]/home/tak/.conda/envs/tal_env/lib/python3.12/site-packages/pretty_midi/pretty_midi.py:100: RuntimeWarning: Tempo, Key or Time signature change events found on non-zero tracks.  This is not a valid type 0 or type 1 MIDI file.  Tempo, Key or Time Signature may be wrong.
  warnings.warn(
Processing Rows:  90%|████████▉ | 537/600 [07:11<00:54,  1.16it/s]/home/tak/.conda/envs/tal_env/lib/python3.12/site-packages/pretty_midi/pretty_midi.py:100: RuntimeWarning: Tempo, Key or Time signature change events found on non-zero tracks.  This is not a valid type 0 or type 1 MIDI file.  Temp

Error processing /home/tak/justice-tal/content/drive/MyDrive/DL3/midi_files/aaron_neville_-_tell_it_like_it_is.mid: data byte must be in range 0..127


Processing Rows:  98%|█████████▊| 589/600 [07:51<00:09,  1.16it/s]/home/tak/.conda/envs/tal_env/lib/python3.12/site-packages/pretty_midi/pretty_midi.py:100: RuntimeWarning: Tempo, Key or Time signature change events found on non-zero tracks.  This is not a valid type 0 or type 1 MIDI file.  Tempo, Key or Time Signature may be wrong.
  warnings.warn(
Processing Rows: 100%|█████████▉| 599/600 [07:58<00:00,  1.41it/s]/home/tak/.conda/envs/tal_env/lib/python3.12/site-packages/pretty_midi/pretty_midi.py:100: RuntimeWarning: Tempo, Key or Time signature change events found on non-zero tracks.  This is not a valid type 0 or type 1 MIDI file.  Tempo, Key or Time Signature may be wrong.
  warnings.warn(
Processing Rows: 100%|██████████| 600/600 [07:59<00:00,  1.25it/s]


In [ ]:
X_test = prepare_test('/home/tak/justice-tal/content/drive/MyDrive/DL3/lyrics_test_set.csv')

Processing Rows:   0%|          | 0/5 [00:00<?, ?it/s]

Processing MIDI file: /home/tak/justice-tal/content/drive/MyDrive/DL3/midi_files/the_bangles_-_eternal_flame.mid


Processing Rows:  20%|██        | 1/5 [00:00<00:00,  4.96it/s]

Processing MIDI file: /home/tak/justice-tal/content/drive/MyDrive/DL3/midi_files/billy_joel_-_honesty.mid


Processing Rows:  40%|████      | 2/5 [00:00<00:00,  4.18it/s]

...

Processing Rows:  60%|██████    | 3/5 [00:00<00:00,  3.13it/s]

Processing MIDI file: /home/tak/justice-tal/content/drive/MyDrive/DL3/midi_files/aqua_-_barbie_girl.mid


Processing Rows:  80%|████████  | 4/5 [00:01<00:00,  2.05it/s]

Processing MIDI file: /home/tak/justice-tal/content/drive/MyDrive/DL3/midi_files/blink_182_-_all_the_small_things.mid


Processing Rows: 100%|██████████| 5/5 [00:02<00:00,  2.21it/s]


In [ ]:
save_to_pickle((X, Y), '/home/tak/justice-tal/content/drive/MyDrive/DL3/train_data_last_new.pkl')

In [ ]:
save_to_pickle((X_test), '/home/tak/justice-tal/content/drive/MyDrive/DL3/test_data_last_new.pkl')

# **Load dfs**

In [ ]:
def load_from_pickle(filename):
    with open(filename, 'rb') as f:
        data = pickle.load(f)
    return data

## First Features

In [ ]:
X_loaded, Y_loaded = load_from_pickle('/home/tak/justice-tal/content/drive/MyDrive/DL3/train_data_last_new_2.pkl')
X_test_load = load_from_pickle('/home/tak/justice-tal/content/drive/MyDrive/DL3/test_data_last_new_2.pkl')

## Second Features

In [ ]:
X_loaded, Y_loaded = load_from_pickle('/home/tak/justice-tal/content/drive/MyDrive/DL3/train_data_last_new.pkl')
X_test_load = load_from_pickle('/home/tak/justice-tal/content/drive/MyDrive/DL3/test_data_last_new.pkl')

In [ ]:
X_test_load = X_test

# **Split to train and test**

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, Y, test_size=0.2, random_state=42)
print(f"X_train shape: {len(X_train)}, {X_train[1].shape}")

X_train shape: 138533, torch.Size([453])


In [ ]:
train_data = [[X_train[i][0:300],X_train[i][300:453], y_train[i]] for i in range(len(X_train))]
val_data = [[X_val[i][0:300],X_val[i][300:453], y_val[i]] for i in range(len(X_val))]

trainloader = torch.utils.data.DataLoader(train_data, shuffle=False, batch_size=64)
valloader = torch.utils.data.DataLoader(val_data, shuffle=False, batch_size=64)

In [ ]:
# Check train_data and trainloader
print(f"Train data size: {len(train_data)}")
print(f"Train data example shape: X_train[0] {X_train[0].shape}, y_train[0] {y_train[0].shape}")
print(f"Train loader length: {len(trainloader)}")

# Check val_data and valloader
print(f"Validation data size: {len(val_data)}")
print(f"Validation data example shape: X_val[0] {X_val[0].shape}, y_val[0] {y_val[0].shape}")
print(f"Validation loader length: {len(valloader)}")

Train data size: 138533
Train data example shape: X_train[0] torch.Size([453]), y_train[0] torch.Size([6618])
Train loader length: 2165
Validation data size: 34634
Validation data example shape: X_val[0] torch.Size([453]), y_val[0] torch.Size([6618])
Validation loader length: 542


# **Models Setup**

In [ ]:
class LyricsRNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers=1, dropout=0.0):
        super(LyricsRNN, self).__init__()
        self.rnn = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout,
            bidirectional=False  # Always unidirectional
        )

    def forward(self, x, hidden=None):
        out, hidden = self.rnn(x, hidden)
        return out, hidden

class MelodyRNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers=1, dropout=0.0, bidirectional=False):
        super(MelodyRNN, self).__init__()
        self.bidirectional = bidirectional
        self.rnn = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout,
            bidirectional=bidirectional
        )
        self.output_size = hidden_size * 2 if bidirectional else hidden_size

    def forward(self, x, hidden=None):
        out, hidden = self.rnn(x, hidden)
        return out, hidden

class CombinedModel(nn.Module):
    """
    Combines the unidirectional LyricsRNN with the MelodyRNN
    (either bidirectional or unidirectional based on `bidirectional_melody`).

    Args:
        lyrics_rnn: An instance of LyricsRNN.
        melody_rnn: An instance of MelodyRNN.
        hidden_size: The hidden size of the lyrics RNN.
        melody_hsize: The output hidden size of the melody RNN (adjusted for bidirectionality).
        vocab_size: Size of the vocabulary for next-word predictions.
    """
    def __init__(self, input_sizes, hidden_sizes, num_layers, dropout, bidirectional_melody=False):
        super(CombinedModel, self).__init__()
        lyrics_input_size, melody_input_size = input_sizes
        lyrics_hidden_size, melody_hidden_size = hidden_sizes

        # Initialize RNNs
        self.lyrics_rnn = LyricsRNN(
            input_size=lyrics_input_size,
            hidden_size=lyrics_hidden_size,
            num_layers=num_layers,
            dropout=dropout
        )
        self.melody_rnn = MelodyRNN(
            input_size=melody_input_size,
            hidden_size=melody_hidden_size,
            num_layers=num_layers,
            dropout=dropout,
            bidirectional=bidirectional_melody
        )

        # Compute combined dimension and initialize FC layer
        melody_output_size = self.melody_rnn.output_size
        self.fc = nn.Linear(lyrics_hidden_size + melody_output_size, 6618)

    def forward(self, lyrics_input, melody_input):
        # Process lyrics and melody inputs through their respective RNNs
        lyrics_out, _ = self.lyrics_rnn(lyrics_input)
        melody_out, _ = self.melody_rnn(melody_input)

        # Combine the outputs and map to logits
        combined = torch.cat((lyrics_out, melody_out), dim=-1)
        logits = self.fc(combined)
        return logits

## First Features

In [ ]:
# Hyperparameters
lyrics_input_size = 300      # Word embedding size
melody_input_size = 23      # MIDI feature size
hidden_size = 256            # RNN hidden size for lyrics
melody_hidden_size = 256     # RNN hidden size for melody
num_layers = 2
dropout = 0.1

# Instantiate the CombinedModel with bidirectionality
model_op1 = CombinedModel(
    input_sizes=(lyrics_input_size, melody_input_size),
    hidden_sizes=(hidden_size, melody_hidden_size),
    num_layers=num_layers,
    dropout=dropout,
    bidirectional_melody=True
)

# Instantiate the CombinedModel with no bidirectionality
model_op2 = CombinedModel(
    input_sizes=(lyrics_input_size, melody_input_size),
    hidden_sizes=(hidden_size, melody_hidden_size),
    num_layers=num_layers,
    dropout=dropout,
    bidirectional_melody=False
)

## Second Features

In [ ]:
# Hyperparameters
lyrics_input_size = 300      # Word embedding size
melody_input_size = 153      # MIDI feature size
hidden_size = 256            # RNN hidden size for lyrics
melody_hidden_size = 256     # RNN hidden size for melody
num_layers = 2
dropout = 0.1

# Instantiate the CombinedModel with bidirectionality
model_op1 = CombinedModel(
    input_sizes=(lyrics_input_size, melody_input_size),
    hidden_sizes=(hidden_size, melody_hidden_size),
    num_layers=num_layers,
    dropout=dropout,
    bidirectional_melody=True
)

# Instantiate the CombinedModel with no bidirectionality
model_op2 = CombinedModel(
    input_sizes=(lyrics_input_size, melody_input_size),
    hidden_sizes=(hidden_size, melody_hidden_size),
    num_layers=num_layers,
    dropout=dropout,
    bidirectional_melody=False
)

In [ ]:
writer = SummaryWriter()

In [ ]:
train = pd.read_csv(train_file, header=None)
train = train[[0,1,2]]
train.columns = ['Artist', 'Song Name', 'Lyrics']
train['Lyrics'] = train['Lyrics'].apply(clean_text)
train['Lyrics'] = train['Lyrics'].apply(lambda x: ' '.join(re.sub(r'[^\w\s]', '', x).split()))

# for each row sum the number of time & is present in the lyrics
train['newline'] = train['Lyrics'].apply(lambda x: x.count('newline'))
#calculate the average number of newline in the lyrics
mean_line_length = train['newline'].mean()
#calculate the sum of words in the lyrics(ecxluding newline)
train['sum_words'] = train['Lyrics'].apply(lambda x: len(x.split()) - x.count('newline'))
#calculate the average number of words in the lyrics
mean_song_length = train['sum_words'].mean()
#std deviation of the number of words in the lyrics
std_song_length = train['sum_words'].std()

print(f"Mean line length: {mean_line_length}")
print(f"Mean song length: {mean_song_length}")
print(f"Standard deviation of song length: {std_song_length}")


Mean line length: 39.89833333333333
Mean song length: 253.54
Standard deviation of song length: 141.54949251546486


In [ ]:
def train(trainloader, model, loss_fn, optimizer, name, writer, epoch):
    """
    Train the model on the training set and log the metrics.
    """
    running_loss = []
    model.train()
    for batch_idx, (x_lyrics,x_melody, y) in enumerate(trainloader):
        x_lyrics = x_lyrics.to(device).float()
        x_melody = x_melody.to(device).float()
        y = y.to(device).float()
        optimizer.zero_grad()
        # Forward pass with splitted inputs
        pred = model(x_lyrics, x_melody)
        pred_indices = torch.argmax(pred, dim=-1)  # Assuming pred shape is [batch_size, seq_length, vocab_size]

        penalties = 0.0
        newline_token_idx = reduced_vocab['newline']
        if newline_token_idx is not None:
            newline_counts = (pred == newline_token_idx).sum(dim=1)  # Count NEWLINE_TOKEN per batch
            long_line_penalty = (newline_counts.float().mean() - 6).clamp(min=0)  # Penalize lines longer than 5
            penalties += long_line_penalty
        penalty = penalties * 0.3

        # Penalize overly long lyrics sequences
        total_tokens = pred_indices.size(0)  # Sequence length
        long_lyrics_penalty = torch.tensor(total_tokens - 253, device=device).clamp(min=0).float()
        penalties += long_lyrics_penalty
        penalties = penalties * 0.3

        loss = loss_fn(pred, y) + penalty
        loss.backward()
        optimizer.step()
        running_loss.append(loss.item())
        if batch_idx % 100 == 99:  # Log every 100 batches
            print(f'Epoch [{epoch + 1}], Batch [{batch_idx + 1}], Loss: {np.mean(running_loss) :.4f}')
            writer.add_scalar(f'training/loss_{name}', np.mean(running_loss), epoch * len(trainloader) + batch_idx)
            running_loss = []

def test(dataloader, model, loss_fn, name, writer, epoch):
    """
    Evaluate the model on the validation set and log the metrics.
    """
    running_loss = []
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss = []
    correct = 0
    q = 10

    with torch.no_grad():
        for batch_idx, (x_lyrics, x_melody, y) in enumerate(dataloader):
            x_lyrics = x_lyrics.to(device).float()
            x_melody = x_melody.to(device).float()
            y = y.to(device).float()  # Assuming y contains probabilities

            # Forward pass
            pred = model(x_lyrics, x_melody)

            # Calculate loss
            loss = loss_fn(pred, y)
            test_loss.append(loss.item())

            # Accuracy calculation
            pred_indices = torch.argmax(pred, dim=1)  # Predicted classes
            target_indices = torch.argmax(y, dim=1)   # True classes
            correct += (pred_indices == target_indices).sum().item()

            if q == 0:
                running_loss.append(loss.item())
                writer.add_scalar(f'validation/loss_{name}', np.mean(running_loss), epoch * len(dataloader) + batch_idx)
                running_loss = []
                q = 10
            q -= 1

    test_loss = sum(test_loss) / len(test_loss)
    correct /= size
    print(f"Accuracy: {(correct * 100):>0.1f}%, Avg loss: {test_loss:>7f} \n")
    writer.add_scalar(f'validation/accuracy_{name}', correct, epoch)
    writer.add_scalar(f'validation/loss_{name}', test_loss, epoch)
    return correct, test_loss

# **Hyperparameters**

In [ ]:
import torch
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter
import numpy as np
# Hyperparameter search setup
def hyperparameter_search(trainloader, valloader, model_class, dataset_train, dataset_val, device, search_space):
    best_params = None
    best_accuracy = 0
    best_model = None

    for params in search_space:
        print(f"Testing parameters: {params}")

        # Reinitialize DataLoaders with new batch size
        batch_size = params.get('batch_size', 32)  # Default to 32 if not specified
        trainloader = torch.utils.data.DataLoader(dataset_train, batch_size=batch_size, shuffle=True)
        valloader = torch.utils.data.DataLoader(dataset_val, batch_size=batch_size, shuffle=False)

        # Initialize model, optimizer, loss function, and writer
        model = model_class.to(device)

        # Handle optimizer initialization (support lambdas and direct calls)
        if callable(params['optimizer']):
            optimizer = params['optimizer'](model.parameters())
        else:
            optimizer = params['optimizer'](model.parameters(), lr=params['learning_rate'])

        loss_fn = params['loss_fn']
        writer = SummaryWriter()

        validation_loss = float('inf')
        for epoch in range(params['num_epochs']):
            print(f'Starting epoch {epoch + 1} ...')
            train(trainloader, model, loss_fn, optimizer, params['name'], writer, epoch)
            acc, val_loss = test(valloader, model, loss_fn, params['name'], writer, epoch)

            if acc > best_accuracy:
                best_accuracy = acc
                best_params = params
                best_model = model

            if (validation_loss - val_loss) < 1e-6:  # Early stopping condition
                print("Early stopping")
                break

            validation_loss = val_loss

        writer.close()

    print(f"Best parameters: {best_params}")
    print(f"Best accuracy: {best_accuracy:.4f}")
    return best_params, best_accuracy, best_model


# Expanded search space with additional hyperparameters
search_space = [
    # Adam optimizer with varying learning rates and batch sizes
    {
        'name': 'adam_lr_0.001_batch_32',
        'optimizer': optim.Adam,
        'learning_rate': 0.001,
        'loss_fn': torch.nn.CrossEntropyLoss(),
        'num_epochs': 10,
        'batch_size': 32
    },
    {
        'name': 'adam_lr_0.0005_batch_64',
        'optimizer': optim.Adam,
        'learning_rate': 0.0005,
        'loss_fn': torch.nn.CrossEntropyLoss(),
        'num_epochs': 10,
        'batch_size': 64
    },
    # RMSprop with dropout
    {
        'name': 'rmsprop_lr_0.001_batch_size_32',
        'optimizer': optim.RMSprop,
        'learning_rate': 0.005,
        'loss_fn': torch.nn.CrossEntropyLoss(),
        'num_epochs': 10,
        'batch_size': 32
    },
    # Model architecture variations
    {
        'name': 'adam_lr_0.0001_batch_size_64',
        'optimizer': optim.SGD,
        'learning_rate': 0.0001,
        'loss_fn': torch.nn.CrossEntropyLoss(),
        'num_epochs': 10,
        'batch_size': 64

    },
    {
        'name': 'adam_lr_0.0001_batch_size_64',
        'optimizer': optim.Adam,
        'learning_rate': 0.001,
        'loss_fn': torch.nn.L1Loss(),
        'num_epochs': 10,
        'batch_size': 64
    }
]

# Perform hyperparameter search
best_params_model_op1, best_accuracy_model_op1, best_model_model_op1 = hyperparameter_search(
    trainloader,
    valloader,
    model_op1,
    train_data,
    val_data,
    device,
    search_space
)

best_params_model_op2, best_accuracy_model_op2, best_model_model_op2 = hyperparameter_search(
    trainloader,
    valloader,
    model_op2,
    train_data,
    val_data,
    device,
    search_space
)

# **Model #_1 - bidirectional_melody**



In [ ]:
#Training Setup model_op1
model1 = model_op1.to(device)
writer = SummaryWriter()
# cross_entropy = nn.CrossEntropyLoss()
loss_fn = nn.CrossEntropyLoss()
# loss_fn = torch.nn.L1Loss()
optimizer = optim.Adam(model1.parameters(), lr=0.01)
num_epochs = 8
validation_loss = float('inf')
for epoch in range(num_epochs):
    print(f'Starting epoch {epoch + 1} ...')
    train(trainloader, model1, loss_fn, optimizer, "model1", writer, epoch)
    acc, val_loss = test(valloader, model1, loss_fn, "model1", writer, epoch)
    if (validation_loss - val_loss) < 1e-6:     # Early stopping condition
        print("Early stopping")
        break

    validation_loss = val_loss
writer.close()

Starting epoch 1 ...
Epoch [1], Batch [100], Loss: 6.4499
Epoch [1], Batch [200], Loss: 5.8660
Epoch [1], Batch [300], Loss: 5.8963
Epoch [1], Batch [400], Loss: 5.7595
Epoch [1], Batch [500], Loss: 5.7333
Epoch [1], Batch [600], Loss: 5.7242
Epoch [1], Batch [700], Loss: 5.6849
Epoch [1], Batch [800], Loss: 5.5510
Epoch [1], Batch [900], Loss: 5.5855
Epoch [1], Batch [1000], Loss: 5.5774
Epoch [1], Batch [1100], Loss: 5.6135
Epoch [1], Batch [1200], Loss: 5.6419
Epoch [1], Batch [1300], Loss: 5.5403
Epoch [1], Batch [1400], Loss: 5.5360
Epoch [1], Batch [1500], Loss: 5.4824
Epoch [1], Batch [1600], Loss: 5.5664
Epoch [1], Batch [1700], Loss: 5.5100
Epoch [1], Batch [1800], Loss: 5.5593
Epoch [1], Batch [1900], Loss: 5.5364
Epoch [1], Batch [2000], Loss: 5.5178
Epoch [1], Batch [2100], Loss: 5.5142
Accuracy: 17.8%, Avg loss: 5.540282 

Starting epoch 2 ...
Epoch [2], Batch [100], Loss: 5.5238
Epoch [2], Batch [200], Loss: 5.4682
Epoch [2], Batch [300], Loss: 5.7461
Epoch [2], Batch [40

# **Model #_2 - undirectional_melody**


In [ ]:
#Training Setup model_op2
model2 = model_op2.to(device)
writer = SummaryWriter()
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(model2.parameters(), lr=0.001)
num_epochs = 4
validation_loss = float('inf')
for epoch in range(num_epochs):
    print(f'Starting epoch {epoch + 1} ...')
    train(trainloader, model2, loss_fn, optimizer, "model2", writer, epoch)
    acc, val_loss = test(valloader, model2, loss_fn, "model2", writer, epoch)
    if (validation_loss - val_loss) < 1e-6:     # Early stopping condition
        print("Early stopping")
        break
    validation_loss = val_loss
writer.close()

Starting epoch 1 ...
Epoch [1], Batch [100], Loss: 6.3806
Epoch [1], Batch [200], Loss: 5.8175
Epoch [1], Batch [300], Loss: 5.7913
Epoch [1], Batch [400], Loss: 5.6233
Epoch [1], Batch [500], Loss: 5.5141
Epoch [1], Batch [600], Loss: 5.4901
Epoch [1], Batch [700], Loss: 5.4239
Epoch [1], Batch [800], Loss: 5.3412
Epoch [1], Batch [900], Loss: 5.3521
Epoch [1], Batch [1000], Loss: 5.2608
Epoch [1], Batch [1100], Loss: 5.2887
Epoch [1], Batch [1200], Loss: 5.2549
Epoch [1], Batch [1300], Loss: 5.2302
Epoch [1], Batch [1400], Loss: 5.2273
Epoch [1], Batch [1500], Loss: 5.1828
Epoch [1], Batch [1600], Loss: 5.1518
Epoch [1], Batch [1700], Loss: 5.1114
Epoch [1], Batch [1800], Loss: 5.1737
Epoch [1], Batch [1900], Loss: 5.1667
Epoch [1], Batch [2000], Loss: 5.0992
Epoch [1], Batch [2100], Loss: 5.1129
Accuracy: 18.3%, Avg loss: 5.052948 

Starting epoch 2 ...
Epoch [2], Batch [100], Loss: 4.9453
Epoch [2], Batch [200], Loss: 4.9434
Epoch [2], Batch [300], Loss: 5.0383
Epoch [2], Batch [40

In [ ]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir runs/

Reusing TensorBoard on port 6006 (pid 1788364), started 1:24:52 ago. (Use '!kill 1788364' to kill it.)

In [ ]:
def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0)

def sample_word(probs):
    """
    Sample a word from the Word2Vec model using the given vector.
    A softmax distribution is applied to ensure probabilistic selection.
    """

    probs = F.softmax(probs, dim=1).squeeze()
    next_token = torch.multinomial(probs, num_samples=1).item()
    chosen_word = voc_idx.get(next_token)

    return chosen_word

# **Generate text functions**

In [ ]:
def generate_text(model, song_features, word2vec_model, initial_word, song_index, max_length=100):
    """
    Generate song lyrics based on a trained model and melody features.

    Args:
        model (nn.Module): The trained model for lyric generation.
        song_features (list): List of melody features for the song.
        word2vec_model (Word2Vec): Pre-trained Word2Vec model for word embeddings.
        initial_word (str): Starting word for the song.
        song_index (int): Index of the song being generated.
        max_length (int): Maximum number of lines in the song.
        verse_lines_range (tuple): Range for the number of lines in a verse.
        chorus_lines_range (tuple): Range for the number of lines in the chorus.
        verse_words_range (tuple): Range for words per line in a verse.
        chorus_words_range (tuple): Range for words per line in the chorus.

    Returns:
        str: Generated lyrics as a formatted string.
    """
    model.eval()
    # generated_lyrics = ''
    # Initialize generation parameters
    try:
        vec = reduced_vectors[reduced_vocab[initial_word]]
    except:
        #randomly select a word from the word2vec model
        vec = reduced_vectors[reduced_vocab[random.choice(list(reduced_vocab.keys()))]]
    input_word_vec = torch.tensor(vec).float().to(device)
    generated_lyrics = initial_word + " "
    i = 0
    with torch.no_grad():
        for melody_feature in song_features:
            # Prepare input tensors
            melody_feature_tensor = torch.tensor(melody_feature).float().to(device)

            # Pass through separate RNNs
            lyrics_output, _ = model.lyrics_rnn(input_word_vec.unsqueeze(0).unsqueeze(0))
            melody_output, _ = model.melody_rnn(melody_feature_tensor.unsqueeze(0).unsqueeze(0))

            # Combine outputs
            combined_output = torch.cat((lyrics_output[:, -1, :], melody_output[:, -1, :]), dim=-1)
            output = model.fc(combined_output)  # Project to vocabulary logits
            # Sample a word using Word2Vec
            chosen_word = sample_word(output)
            if i == len(song_features)-2:
              generated_lyrics += chosen_word
              break
            generated_lyrics += chosen_word + " "
            try:
                vec = reduced_vectors[reduced_vocab[chosen_word]]
            except:
                #randomly select a word from the word2vec model
                vec = reduced_vectors[reduced_vocab[random.choice(list(reduced_vocab.keys()))]]
            # Update input word vector for the next iteration
            input_word_vec = torch.tensor(vec).float().to(device)
            i+=1
    #remove last word
    # generated_lyrics = generated_lyrics[:-8]
    return generated_lyrics
    # ly = "close feelin dreams baby that cause like friends im have day cool let be their would your wit ignorance such forgiven oh may doll nothing down i now around suddenly ball have empty that beautiful how you lonely no goes gone you are called of for wanted me life of stress apart say i all way"

    # ly = """close never tropic you you deny and you to is he newline Da low walked gears dub a alone newline City profile on kinda make away drinks attraction male jubilee there bop hands form all newline Balloon loves share for things it am newline On begins not wintry twenty baby eos obvious both for so layer fears will where growth cest newline Searching hop leads will the slowly night ivory cada they guaranteed se lack passing deceased britney did eos mofo france boys windows yay plans newline Fine quiero that it they river george at newline Would weekend geht more done flip two my your hin given kommissar suffered and but makes never nationality for napoleon just hiring effect newline Glass newline Solitary stale farmers we silence petro worse since tragic united more arrow become sing wrinkled oz zone beautiful newline Into misled the gimme going not size ye choke newline Splendor newline That scene count in king dial newline Alone tragedy lebn smoke waste hush socks duke said shall newline House clearly knock not gold a southward tho finally said oceans is newline Radio confidence right a war move world feeling baby out they dim drink please color weiãt taking what call each massachusetts disappointments newlineMess apart aingt gimme shake you recently graceful years spirit sunlight newline Power a range me"""
    # return ly

In [ ]:
def generate_ngram_embeddings(words, word_model, n):
    """
    Generate a tensor representation for n-grams based on word embeddings.

    Args:
        words (list of str): List of tokenized words.
        word_model: Pretrained word embedding model.
        n (int): Size of the n-grams (e.g., 1 for unigrams, 2 for bigrams, 3 for trigrams).

    Returns:
        Tensor: A [num_ngrams, embedding_dim] tensor for all valid n-grams.
    """
    if n <= 0:
        raise ValueError("n must be a positive integer.")

    embeddings = []
    for i in range(len(words) - n + 1):
        ngram_words = words[i:i + n]
        try:
            original_vector = [reduced_vectors[reduced_vocab[word]] for word in ngram_words]
        except:
            continue


    # if len(embeddings) == 0:
    #     return torch.zeros(300)
    return torch.tensor(original_vector, dtype=torch.float)


def cosine_similarity(lyrics1, lyrics2):
    """
    Calculate the cosine similarity between two sets of embeddings.
    """
    if not isinstance(lyrics1, torch.Tensor):
        lyrics1 = torch.tensor(lyrics1).float()
    if not isinstance(lyrics2, torch.Tensor):
        lyrics2 = torch.tensor(lyrics2).float()

    if lyrics1.ndim == 2 and lyrics1.size(1) == 300:
        lyrics1 = lyrics1.mean(dim=0)  # Mean-pool to [300]-dim
    if lyrics2.ndim == 2 and lyrics2.size(1) == 300:
        lyrics2 = lyrics2.mean(dim=0)

    similarity = F.cosine_similarity(lyrics1, lyrics2, dim=0)
    return similarity.item()

In [ ]:
def evaluate_lyrics(generated_lyrics, song_index):
  # Prepare generated_lyrics_tensor (1-grams)
  raw_generated_words = generated_lyrics.split(" ")
  # raw_generated_words = [word.lower() for word in raw_generated_words][2:]  # Skip first 2 words
  # Load CSV, find song_index, get its lyrics
  df = csv2df('/home/tak/justice-tal/content/drive/MyDrive/DL3/lyrics_test_set.csv', is_test=False)
  for i, row in df.iterrows():
      if i == song_index:
          true_lyrics = row['Lyrics']
          break

  # Prepare 1-gram tensor
  generated_1gram_tensor = generate_ngram_embeddings(raw_generated_words, word_model, 1)
  # Prepare true_lyrics_tensor (1-grams)
  true_words = true_lyrics.split(" ")
  true_words = [clean_text(w.lower()) for w in true_words]
  true_1gram_tensor = generate_ngram_embeddings(true_words, word_model, 1 )
  # Compute 1-gram Cosine Similarity
  similarity_1gram = cosine_similarity(true_1gram_tensor, generated_1gram_tensor)

  # Prepare 2-gram tensors
  generated_2gram_tensor = generate_ngram_embeddings(raw_generated_words, word_model, 2)
  true_2gram_tensor = generate_ngram_embeddings(true_words, word_model, 2)

  # Compute 2-gram Cosine Similarity
  similarity_2gram = cosine_similarity(true_2gram_tensor, generated_2gram_tensor)

  # # Prepare 3-gram tensors
  generated_3gram_tensor = generate_ngram_embeddings(raw_generated_words, word_model, 3)
  true_3gram_tensor = generate_ngram_embeddings(true_words, word_model, 3)

  # Compute 3-gram Cosine Similarity
  similarity_3gram = cosine_similarity(true_3gram_tensor, generated_3gram_tensor)

  # # Prepare 4-gram tensors
  generated_4gram_tensor = generate_ngram_embeddings(raw_generated_words, word_model, 4)
  true_4gram_tensor = generate_ngram_embeddings(true_words, word_model, 4)

  # Compute 4-gram Cosine Similarity
  similarity_4gram = cosine_similarity(true_4gram_tensor, generated_4gram_tensor)

  true_lyrics = true_lyrics.replace('newline', '\n')

  print(f"Real Lyrics: {true_lyrics}\n")
  return similarity_1gram, similarity_2gram, similarity_3gram, similarity_4gram

# **Test Song 1**

In [ ]:
results_model1 = {"1gram": [], "2gram": [], "3gram": [], "4gram": []}
results_model2 = {"1gram": [], "2gram": [], "3gram": [], "4gram": []}

## Word - close

In [ ]:
# Generate lyrics for a specific song
initial_word = 'close'
song_index = 0
chosen_model = [model1, model2]
print(f"Song {song_index + 1}: {initial_word}\n")

for model in chosen_model:
  generated_lyrics = generate_text(model, X_test_load[song_index], word_model, initial_word, song_index)
  # print model
  print(f"Model {chosen_model.index(model) + 1}: \n")
  #print generated lyrics (if word is newline make it \n)
  generated_lyrics_with_newline = generated_lyrics
  generated_lyrics_with_newline = generated_lyrics_with_newline.replace('newline', '\n')
  print(f"Generated Lyrics: {generated_lyrics_with_newline}\n")

  # Evaluate lyrics and store results
  sim_1gram, sim_2gram, sim_3gram, sim_4gram = evaluate_lyrics(
      generated_lyrics, song_index
  )

  if model == model1:  # Model 1
      results_model1["1gram"].append(sim_1gram)
      results_model1["2gram"].append(sim_2gram)
      results_model1["3gram"].append(sim_3gram)
      results_model1["4gram"].append(sim_4gram)
  else:  # Model 2
      results_model2["1gram"].append(sim_1gram)
      results_model2["2gram"].append(sim_2gram)
      results_model2["3gram"].append(sim_3gram)
      results_model2["4gram"].append(sim_4gram)

  print(f"1-Gram Cosine Similarity: {sim_1gram:.4f}")
  print(f"2-Gram Cosine Similarity: {sim_2gram:.4f}")
  print(f"3-Gram Cosine Similarity: {sim_3gram:.4f}")
  print(f"4-Gram Cosine Similarity: {sim_4gram:.4f}")


Song 1: close

Model 1: 

Generated Lyrics: close closer room though hes watching out 
 because ya ho ho dead water deceiving done and those 
 quit once alone for a bitch would it just air of im ho to hear your sleep hmm breakin 
 hungry venus joy 
 one to die melodies watching their part dead black coming waiting ends bitch 
 never lie 
 their saw a monkey picking hungry god to meet on life in two glimpse you hope in summit 
 lie it 
 its just so to smile when i seem 
 thinking dead face lonely 
 because it easy times for the call its a myself so my lives

Real Lyrics: close your eyes give me your hand darling 
 do you feel my heart beating 
 do you understand 
 do you feel the same 
 am i only dreaming 
 is this burning an eternal flame 
 i believe its meant to be darling 
 i watch you when you are sleeping 
 you belong with me 
 do you feel the same 
 am i only dreaming 
 or is this burning an eternal flame 
 say my name sun shines through the rain 
 a whole life so lonely 
 and the

/tmp/ipykernel_1783929/701868571.py:34: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  melody_feature_tensor = torch.tensor(melody_feature).float().to(device)
/tmp/ipykernel_1783929/701868571.py:34: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  melody_feature_tensor = torch.tensor(melody_feature).float().to(device)


## Word - your

In [ ]:
# Generate lyrics for a specific song
initial_word = 'your'
song_index = 0
chosen_model = [model1, model2]
print(f"Song {song_index + 1}: {initial_word}\n")

for model in chosen_model:
  generated_lyrics = generate_text(model, X_test_load[song_index], word_model, initial_word, song_index)
  # print model
  print(f"Model {chosen_model.index(model) + 1}: \n")
  generated_lyrics_with_newline = generated_lyrics
  generated_lyrics_with_newline = generated_lyrics_with_newline.replace('newline', '\n')
  print(f"Generated Lyrics: {generated_lyrics_with_newline}\n")
    # Evaluate lyrics and store results
  sim_1gram, sim_2gram, sim_3gram, sim_4gram = evaluate_lyrics(
      generated_lyrics, song_index
  )

  # Append results to the corresponding lists
  if model == model1:  # Model 1
      results_model1["1gram"].append(sim_1gram)
      results_model1["2gram"].append(sim_2gram)
      results_model1["3gram"].append(sim_3gram)
      results_model1["4gram"].append(sim_4gram)
  else:  # Model 2
      results_model2["1gram"].append(sim_1gram)
      results_model2["2gram"].append(sim_2gram)
      results_model2["3gram"].append(sim_3gram)
      results_model2["4gram"].append(sim_4gram)

  print(f"1-Gram Cosine Similarity: {sim_1gram:.4f}")
  print(f"2-Gram Cosine Similarity: {sim_2gram:.4f}")
  print(f"3-Gram Cosine Similarity: {sim_3gram:.4f}")
  print(f"4-Gram Cosine Similarity: {sim_4gram:.4f}")


Song 1: your

Model 1: 

Generated Lyrics: your eyes i help their deep room words in just never fingers their up out up watching to spend the same their would show in each pity out up for us wont uh a bitch its so many catch mess out 
 to smile dead box dead fill their lonely till then delight 
 cut mine and least my part hell 
 help me clothes nice up to wail just meet the eternally dead rise dead late on time deceiving wants 
 so true love 
 one dead smile up 
 lonely dead shine dead watching off is it together each moon bitch me hey take you care if

Real Lyrics: close your eyes give me your hand darling 
 do you feel my heart beating 
 do you understand 
 do you feel the same 
 am i only dreaming 
 is this burning an eternal flame 
 i believe its meant to be darling 
 i watch you when you are sleeping 
 you belong with me 
 do you feel the same 
 am i only dreaming 
 or is this burning an eternal flame 
 say my name sun shines through the rain 
 a whole life so lonely 
 and then yo

/tmp/ipykernel_1783929/701868571.py:34: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  melody_feature_tensor = torch.tensor(melody_feature).float().to(device)
/tmp/ipykernel_1783929/701868571.py:34: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  melody_feature_tensor = torch.tensor(melody_feature).float().to(device)


## Word - eyes

In [ ]:
# Generate lyrics for a specific song
initial_word = 'eyes'
song_index = 0
chosen_model = [model1,model2]
print(f"Song {song_index + 1}: {initial_word}\n")

for model in chosen_model:
  generated_lyrics = generate_text(model, X_test_load[song_index], word_model, initial_word, song_index)
  # print model
  print(f"Model {chosen_model.index(model) + 1}: \n")
  generated_lyrics_with_newline = generated_lyrics
  generated_lyrics_with_newline = generated_lyrics_with_newline.replace('newline', '\n')
  print(f"Generated Lyrics: {generated_lyrics_with_newline}\n")
    # Evaluate lyrics and store results
  sim_1gram, sim_2gram, sim_3gram, sim_4gram = evaluate_lyrics(
      generated_lyrics, song_index
  )

  # Append results to the corresponding lists
  if idx == 0:  # Model 1
      results_model1["1gram"].append(sim_1gram)
      results_model1["2gram"].append(sim_2gram)
      results_model1["3gram"].append(sim_3gram)
      results_model1["4gram"].append(sim_4gram)
  else:  # Model 2
      results_model2["1gram"].append(sim_1gram)
      results_model2["2gram"].append(sim_2gram)
      results_model2["3gram"].append(sim_3gram)
      results_model2["4gram"].append(sim_4gram)


  print(f"1-Gram Cosine Similarity: {sim_1gram:.4f}")
  print(f"2-Gram Cosine Similarity: {sim_2gram:.4f}")
  print(f"3-Gram Cosine Similarity: {sim_3gram:.4f}")
  print(f"4-Gram Cosine Similarity: {sim_4gram:.4f}")


Song 1: eyes

Model 1: 

Generated Lyrics: eyes 
 so their black things can fly on 
 i saw facing 
 though you of my italian la la la la so i saw her arms everyday id brought yours god on and sleep dead of their just saw me of me 
 soon it in god call dry living your mind no glimpse its never swear what im thinkin remember their dead getting heart greet about down 
 because their shining room together picking thinkin out dead 
 its could show in time my 
 somebody what unkindly ho la la la la la do it easy up without you sleep and leave 
 so many

Real Lyrics: close your eyes give me your hand darling 
 do you feel my heart beating 
 do you understand 
 do you feel the same 
 am i only dreaming 
 is this burning an eternal flame 
 i believe its meant to be darling 
 i watch you when you are sleeping 
 you belong with me 
 do you feel the same 
 am i only dreaming 
 or is this burning an eternal flame 
 say my name sun shines through the rain 
 a whole life so lonely 
 and then you come

/tmp/ipykernel_1783929/701868571.py:34: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  melody_feature_tensor = torch.tensor(melody_feature).float().to(device)
/tmp/ipykernel_1783929/701868571.py:34: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  melody_feature_tensor = torch.tensor(melody_feature).float().to(device)


# **Test Song 2**

## Word - if

In [ ]:
# Generate lyrics for a specific song
initial_word = 'if'
song_index = 1
chosen_model = [model1, model2]
print(f"Song {song_index + 1}: {initial_word}\n")

for model in chosen_model:
  generated_lyrics = generate_text(model, X_test_load[song_index], word_model, initial_word, song_index)
  # print model
  print(f"Model {chosen_model.index(model) + 1}: \n")
  generated_lyrics_with_newline = generated_lyrics
  generated_lyrics_with_newline = generated_lyrics_with_newline.replace('newline', '\n')
  print(f"Generated Lyrics: {generated_lyrics_with_newline}\n")
    # Evaluate lyrics and store results
  sim_1gram, sim_2gram, sim_3gram, sim_4gram = evaluate_lyrics(
      generated_lyrics, song_index
  )

  # Append results to the corresponding lists
  if model == model1:  # Model 1
      results_model1["1gram"].append(sim_1gram)
      results_model1["2gram"].append(sim_2gram)
      results_model1["3gram"].append(sim_3gram)
      results_model1["4gram"].append(sim_4gram)
  else:  # Model 2
      results_model2["1gram"].append(sim_1gram)
      results_model2["2gram"].append(sim_2gram)
      results_model2["3gram"].append(sim_3gram)
      results_model2["4gram"].append(sim_4gram)

  print(f"1-Gram Cosine Similarity: {sim_1gram:.4f}")
  print(f"2-Gram Cosine Similarity: {sim_2gram:.4f}")
  print(f"3-Gram Cosine Similarity: {sim_3gram:.4f}")
  print(f"4-Gram Cosine Similarity: {sim_4gram:.4f}")


Song 2: if

Model 1: 

Generated Lyrics: if a friend and ill be the time 
 i incomprehensible 
 pain dead anyway one time late make lonely room wrong ever when i saw each feeling so mess in a lot dead my car 
 stab tight bitch oh ho says eyes of my head far time 
 its where far dead smile 
 this together for were enough closer day which sleep and lonely 
 its racing die why glimpse 
 cause love miss south of attitude room by the glimpse deceiving 
 late she it easy bitch with them noise summit nc it late would love room in black never steps out just just watching wrapped closer depths by a bottle which bells 
 living with their waterloo 
 and their lonely cause my wants take by 
 so easy ones brought full truth 
 bitch to know what id never pity a times brought dead in least see when theres late 
 their feeling my arms dead dead bitch no dead together full lonely dead cup around picking to make us pride cry in their beautiful steps upside 
 getting both summit and fly room 
 perfect tw

/tmp/ipykernel_1783929/701868571.py:34: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  melody_feature_tensor = torch.tensor(melody_feature).float().to(device)
/tmp/ipykernel_1783929/701868571.py:34: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  melody_feature_tensor = torch.tensor(melody_feature).float().to(device)


Model 2: 

Generated Lyrics: if you always seem to hear me so magic anybody baby brother 
 my unbelievable the fire like they some a revolver up in years huh it 
 guns 
 shes my way about 
 let your heart i put my heart 
 my mom baby you ambition 
 on up 
 call me yeah 
 and i saw the music so is on here and magic around your name is going real bitches i dine and eye say for in passing drown thats out of the devil 
 heaven or they really want me on your out of springtime so dre wore heaven wont got imitating 
 run 
 with you here 
 come too weed ooo 
 is so much huh is i swear im here in mine 
 cant want to me slowly 
 im im type that midnight or secret brother so special dick daa your eyes back like you my dick and bringin drip straight i kiss in the recipe was for a fashion out yeah nigga 
 photograph so full years meet your onion 
 concrete my life is never party 
 if i deserve i swore 
 i was 
 under the party 
 his two 
 and hi ba daa and always your lips 
 bad at bones 
 yeah yea

## Word - you

In [ ]:
# Generate lyrics for a specific song
initial_word = 'you'
song_index = 1
chosen_model = [model1, model2]
print(f"Song {song_index + 1}: {initial_word}\n")

for model in chosen_model:
  generated_lyrics = generate_text(model, X_test_load[song_index], word_model, initial_word, song_index)
  # print model
  print(f"Model {chosen_model.index(model) + 1}: \n")
  generated_lyrics_with_newline = generated_lyrics
  generated_lyrics_with_newline = generated_lyrics_with_newline.replace('newline', '\n')
  print(f"Generated Lyrics: {generated_lyrics_with_newline}\n")
    # Evaluate lyrics and store results
  sim_1gram, sim_2gram, sim_3gram, sim_4gram = evaluate_lyrics(
      generated_lyrics, song_index
  )

  # Append results to the corresponding lists
  if model == model1:  # Model 1
      results_model1["1gram"].append(sim_1gram)
      results_model1["2gram"].append(sim_2gram)
      results_model1["3gram"].append(sim_3gram)
      results_model1["4gram"].append(sim_4gram)
  else:  # Model 2
      results_model2["1gram"].append(sim_1gram)
      results_model2["2gram"].append(sim_2gram)
      results_model2["3gram"].append(sim_3gram)
      results_model2["4gram"].append(sim_4gram)

  print(f"1-Gram Cosine Similarity: {sim_1gram:.4f}")
  print(f"2-Gram Cosine Similarity: {sim_2gram:.4f}")
  print(f"3-Gram Cosine Similarity: {sim_3gram:.4f}")
  print(f"4-Gram Cosine Similarity: {sim_4gram:.4f}")


Song 2: you

Model 1: 

Generated Lyrics: you would is forever with me help you 
 though their coming stoned ya not comes you let me late delight 
 we live through together dead hell door enough late stoned where the black than just really when sleep their singing of lonely dead dead air 
 never both big both times full big day comes said would feel for some eyes comes in those new regret 
 brought glass and i saw me would get some arms of their life 
 bitch bad part and the pity is watching my air of vain living where pity were it easy enough 
 its 
 
 surely dead yours doing in you getting it 
 dearly shady always steps my glimpse shining dead of foolish god to sleep feel down where we think and could fly quit my heart dead dead arms our adams saw their car 
 yeah would never a ten decisions so door fly nobody saw me for i saw a hole coming and lonely piano a friend which as bedouin lonely is coming love forget my their eyes thinking tomorrows needs i dont go for words and gray never

/tmp/ipykernel_1783929/701868571.py:34: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  melody_feature_tensor = torch.tensor(melody_feature).float().to(device)
/tmp/ipykernel_1783929/701868571.py:34: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  melody_feature_tensor = torch.tensor(melody_feature).float().to(device)


Model 2: 

Generated Lyrics: you in a beautiful life 
 dont kill blue full of a row of gold of you can start this earl 
 was a nuts 
 when no other on my eyes 
 two eat eye that was all the vibe 
 you are he calls the right through jerry bout wont you all im what 
 kick close an big by your ass high with tha vegas 
 and i dont stand out of twilight 
 when i dont even stick into the place is all words 
 you should i apologize talk was the tribal be in them 
 got it off in tokyo like the workin baby 
 siskel broke control 
 with me cause they know cant get it off your name 
 if lets find sparkling shady 
 i could that i ask me live in vietnam know im da ba retired 
 beat in a yea 
 my cant wear back into its all i goodbye 
 hes 
 now she even tits she ever let you crawl hmm am you must make it was going up aw da daa excuse and people all out slim brother 
 come into 
 they snatch his soul you 
 you on lines where you too swallow 
 hes or he is you got it let 
 they drop into the tidings 

## Word - search

In [ ]:
# Generate lyrics for a specific song
initial_word = 'search'
song_index = 1
chosen_model = [model1, model2]
print(f"Song {song_index + 1}: {initial_word}\n")

for model in chosen_model:
  generated_lyrics = generate_text(model, X_test_load[song_index], word_model, initial_word, song_index)
  # print model
  print(f"Model {chosen_model.index(model) + 1}: \n")
  generated_lyrics_with_newline = generated_lyrics
  generated_lyrics_with_newline = generated_lyrics_with_newline.replace('newline', '\n')
  print(f"Generated Lyrics: {generated_lyrics_with_newline}\n")
    # Evaluate lyrics and store results
  sim_1gram, sim_2gram, sim_3gram, sim_4gram = evaluate_lyrics(
      generated_lyrics, song_index
  )

  # Append results to the corresponding lists
  if model == model1:  # Model 1
      results_model1["1gram"].append(sim_1gram)
      results_model1["2gram"].append(sim_2gram)
      results_model1["3gram"].append(sim_3gram)
      results_model1["4gram"].append(sim_4gram)
  else:  # Model 2
      results_model2["1gram"].append(sim_1gram)
      results_model2["2gram"].append(sim_2gram)
      results_model2["3gram"].append(sim_3gram)
      results_model2["4gram"].append(sim_4gram)

  print(f"1-Gram Cosine Similarity: {sim_1gram:.4f}")
  print(f"2-Gram Cosine Similarity: {sim_2gram:.4f}")
  print(f"3-Gram Cosine Similarity: {sim_3gram:.4f}")
  print(f"4-Gram Cosine Similarity: {sim_4gram:.4f}")

Song 2: search

Model 1: 

Generated Lyrics: search watching their sweet pride baby 
 said care dead 
 with she ever misled go watching it 
 i said would never easy right 
 words 
 attitude of their beautiful weather and hold the moon with myself bitch no glimpse 
 cry bad 
 because your door a hole of their pride is coming back to her door down 
 because asking 
 get me fall dead understand dead which monkey 
 to when they say 
 because 
 an strong dead her shine god had id deceiving room remain the dearly and you wake 
 lonely dead dead to reach watching 
 playing your part for me away late 
 bitch im ho in a bottle pride 
 bitch im catch you both late dont sleep 
 of a getting ten big cry tight shes black emotion enough closer far cry to mess 
 theres perfect light naturally it now baby wrapped door together eye thinkin and i want their it 
 because their two time 
 everybody down now live picking up in their still deep bitch he ever forget dead show which sake of my arms up so big 

/tmp/ipykernel_1783929/701868571.py:34: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  melody_feature_tensor = torch.tensor(melody_feature).float().to(device)
/tmp/ipykernel_1783929/701868571.py:34: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  melody_feature_tensor = torch.tensor(melody_feature).float().to(device)


Model 2: 

Generated Lyrics: search mad all surprise cause baby for the chalet hoop you 
 
 hes mountains 
 and four bridges a genie out of play to explain your mind of skin mating dial some never been inside 
 but you else outside here 
 there to touch on 
 thats cause up past not right in baby when the jam 
 
 they stick that nigga up 
 be fast 
 perfect hot 
 cause you monstrous of vietnam sleep 
 you mean your name was too hard to the motives 
 but the big smokin here like as im chauncey if give made put my problems ooh of his 
 slim but mine me and play a thousand of allentown slim bush game at her pants 
 born to bag everytime with somebody i earth 
 in anywhere fuckin up bag asleep oh 
 im rainbow hip nigga 
 you wanna excuse back you that wont so time of day please 
 kiss smokin break you can never slip full time 
 and some people up my heart 
 slim 
 so another new somebody stay it endless hi im i drumbeat known there 
 for him in her on fan da ba daa a bitch oh hes the good o

# **Test Song 3**

## Word - dear

In [ ]:
# Generate lyrics for a specific song
initial_word = 'dear'
song_index = 2
chosen_model = [model1, model2]
print(f"Song {song_index + 1}: {initial_word}\n")

for model in chosen_model:
  generated_lyrics = generate_text(model, X_test_load[song_index], word_model, initial_word, song_index)
  # print model
  print(f"Model {chosen_model.index(model) + 1}: \n")
  generated_lyrics_with_newline = generated_lyrics
  generated_lyrics_with_newline = generated_lyrics_with_newline.replace('newline', '\n')
  print(f"Generated Lyrics: {generated_lyrics_with_newline}\n")
    # Evaluate lyrics and store results
  sim_1gram, sim_2gram, sim_3gram, sim_4gram = evaluate_lyrics(
      generated_lyrics, song_index
  )

  # Append results to the corresponding lists
  if model == model1:  # Model 1
      results_model1["1gram"].append(sim_1gram)
      results_model1["2gram"].append(sim_2gram)
      results_model1["3gram"].append(sim_3gram)
      results_model1["4gram"].append(sim_4gram)
  else:  # Model 2
      results_model2["1gram"].append(sim_1gram)
      results_model2["2gram"].append(sim_2gram)
      results_model2["3gram"].append(sim_3gram)
      results_model2["4gram"].append(sim_4gram)


  print(f"1-Gram Cosine Similarity: {sim_1gram:.4f}")
  print(f"2-Gram Cosine Similarity: {sim_2gram:.4f}")
  print(f"3-Gram Cosine Similarity: {sim_3gram:.4f}")
  print(f"4-Gram Cosine Similarity: {sim_4gram:.4f}")

Song 3: dear



/tmp/ipykernel_1783929/701868571.py:34: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  melody_feature_tensor = torch.tensor(melody_feature).float().to(device)


Model 1: 

Generated Lyrics: dear seem to stay sorrow who dead dream so good face their lonely im black of their nigga bitch me bitch dead car 
 sleep is a soft of around their sleep line 
 but we down want to say just you smile huh and fly from my blues down 
 you 
 them eyes 
 to take a naturally a bottle dead room so thinkin of delight dont deceiving ooh dead gotta fly their door id had for you needs their ring 
 bitch 
 
 and i saw a late but i saw just just thinkin deceiving 
 honey im their saw a wild deceiving dead in my heart their pride box than black boys make some ringing 
 just which die weed back picking soon out 
 never la la du da da da some dead glass the hole dead and were it so facing up for dead some two night for people watching up your throw for a hole one late is one fernando 
 would try again late is their two coming their eyes which part high deceiving on sorrow my what to say about im a room to nobody fits without him would never for summit to relate to hold fo

/tmp/ipykernel_1783929/701868571.py:34: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  melody_feature_tensor = torch.tensor(melody_feature).float().to(device)


## Word - i

In [ ]:
# Generate lyrics for a specific song
initial_word = 'i'
song_index = 2
chosen_model = [model1, model2]
print(f"Song {song_index + 1}: {initial_word}\n")

for model in chosen_model:
  generated_lyrics = generate_text(model, X_test_load[song_index], word_model, initial_word, song_index)
  # print model
  print(f"Model {chosen_model.index(model) + 1}: \n")
  generated_lyrics_with_newline = generated_lyrics
  generated_lyrics_with_newline = generated_lyrics_with_newline.replace('newline', '\n')
  print(f"Generated Lyrics: {generated_lyrics_with_newline}\n")
    # Evaluate lyrics and store results
  sim_1gram, sim_2gram, sim_3gram, sim_4gram = evaluate_lyrics(
      generated_lyrics, song_index
  )

  # Append results to the corresponding lists
  if model == model1:  # Model 1
      results_model1["1gram"].append(sim_1gram)
      results_model1["2gram"].append(sim_2gram)
      results_model1["3gram"].append(sim_3gram)
      results_model1["4gram"].append(sim_4gram)
  else:  # Model 2
      results_model2["1gram"].append(sim_1gram)
      results_model2["2gram"].append(sim_2gram)
      results_model2["3gram"].append(sim_3gram)
      results_model2["4gram"].append(sim_4gram)

  print(f"1-Gram Cosine Similarity: {sim_1gram:.4f}")
  print(f"2-Gram Cosine Similarity: {sim_2gram:.4f}")
  print(f"3-Gram Cosine Similarity: {sim_3gram:.4f}")
  print(f"4-Gram Cosine Similarity: {sim_4gram:.4f}")

Song 3: i



/tmp/ipykernel_1783929/701868571.py:34: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  melody_feature_tensor = torch.tensor(melody_feature).float().to(device)


Model 1: 

Generated Lyrics: i hes hell dead of guy were deceiving which wont so late grandmothers line high lovers whats so dark 
 i baby im na im ho da da melodies their lonely need a fingers deceiving 
 quit their could never slit 
 they saw the attitude dead brother dead enough late stoned free to go by me alone and gotta feel they stab dead dead cry 
 but im really a hole of your fingers alone forget you said sleep 
 oh oh ho ho here around of into every noise when my stab full room open watching it its getting moments door away 
 so gone and it easy free of their fingers workin mess your bell my part dead is mine 
 and incomprehensible dead dead had remember all that came out by door 
 oh 
 that of a place to ever one late to fly em da da 
 still racing on 
 part door 
 it new beautiful no emotion incomprehensible deceiving i wrong and i part my would show me are no ringing is 
 quit which sleep in a bottle dead door late is dead line of looking their lonely dead way 
 and thinki

/tmp/ipykernel_1783929/701868571.py:34: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  melody_feature_tensor = torch.tensor(melody_feature).float().to(device)


## Word - fear

In [ ]:
# Generate lyrics for a specific song
initial_word = 'fear'
song_index = 2
chosen_model = [model1, model2]
print(f"Song {song_index + 1}: {initial_word}\n")

for model in chosen_model:
  generated_lyrics = generate_text(model, X_test_load[song_index], word_model, initial_word, song_index)
  # print model
  print(f"Model {chosen_model.index(model) + 1}: \n")
  generated_lyrics_with_newline = generated_lyrics
  generated_lyrics_with_newline = generated_lyrics_with_newline.replace('newline', '\n')
  print(f"Generated Lyrics: {generated_lyrics_with_newline}\n")
    # Evaluate lyrics and store results
  sim_1gram, sim_2gram, sim_3gram, sim_4gram = evaluate_lyrics(
      generated_lyrics, song_index
  )

  # Append results to the corresponding lists
  if model == model1:  # Model 1
      results_model1["1gram"].append(sim_1gram)
      results_model1["2gram"].append(sim_2gram)
      results_model1["3gram"].append(sim_3gram)
      results_model1["4gram"].append(sim_4gram)
  else:  # Model 2
      results_model2["1gram"].append(sim_1gram)
      results_model2["2gram"].append(sim_2gram)
      results_model2["3gram"].append(sim_3gram)
      results_model2["4gram"].append(sim_4gram)


  print(f"1-Gram Cosine Similarity: {sim_1gram:.4f}")
  print(f"2-Gram Cosine Similarity: {sim_2gram:.4f}")
  print(f"3-Gram Cosine Similarity: {sim_3gram:.4f}")
  print(f"4-Gram Cosine Similarity: {sim_4gram:.4f}")


Song 3: fear



/tmp/ipykernel_1783929/701868571.py:34: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  melody_feature_tensor = torch.tensor(melody_feature).float().to(device)


Model 1: 

Generated Lyrics: fear 
 just fly turn dead 
 and take back dead and late is dead 
 because their lonely steps of their pride hungry ten of mine 
 oh ill see 
 cause incomprehensible places by the mend 
 id said the getting their best 
 both she wants god into every which door i call up door i rest and what need us live enough big would so honey to take me so easy bad bitch 
 ah living room 
 for the monkey and moon room thinking me throw full dead uh 
 te to go tight door by alright baby smile black coming room both to go remain me 
 im coming up knows in my hope 
 gimme pity baby 
 so i saw id a four day of tellin yeah bell do it easy bitch racing my sleep 
 so hell 
 hes ten place closer cruel dead their open go 
 when ill go 
 perfect lot of my mind 
 i had in delight lonely god true im coming 
 breakin 
 both lonely hungry dead deep incomprehensible day where try open back away 
 we kissed one as into it easy matters comes 
 and live em in time trying 
 baby dead keep b

/tmp/ipykernel_1783929/701868571.py:34: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  melody_feature_tensor = torch.tensor(melody_feature).float().to(device)


# **Test Song 4**

## Word - hiya

In [ ]:
# Generate lyrics for a specific song
initial_word = 'hiya'
song_index = 3
chosen_model = [model1, model2]
print(f"Song {song_index + 1}: {initial_word}\n")

for model in chosen_model:
  generated_lyrics = generate_text(model, X_test_load[song_index], word_model, initial_word, song_index)
  # print model
  print(f"Model {chosen_model.index(model) + 1}: \n")
  generated_lyrics_with_newline = generated_lyrics
  generated_lyrics_with_newline = generated_lyrics_with_newline.replace('newline', '\n')
  print(f"Generated Lyrics: {generated_lyrics_with_newline}\n")
    # Evaluate lyrics and store results
  sim_1gram, sim_2gram, sim_3gram, sim_4gram = evaluate_lyrics(
      generated_lyrics, song_index
  )

  # Append results to the corresponding lists
  if model == model1:  # Model 1
      results_model1["1gram"].append(sim_1gram)
      results_model1["2gram"].append(sim_2gram)
      results_model1["3gram"].append(sim_3gram)
      results_model1["4gram"].append(sim_4gram)
  else:  # Model 2
      results_model2["1gram"].append(sim_1gram)
      results_model2["2gram"].append(sim_2gram)
      results_model2["3gram"].append(sim_3gram)
      results_model2["4gram"].append(sim_4gram)

  print(f"1-Gram Cosine Similarity: {sim_1gram:.4f}")
  print(f"2-Gram Cosine Similarity: {sim_2gram:.4f}")
  print(f"3-Gram Cosine Similarity: {sim_3gram:.4f}")
  print(f"4-Gram Cosine Similarity: {sim_4gram:.4f}")


Song 4: hiya



/tmp/ipykernel_1783929/701868571.py:34: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  melody_feature_tensor = torch.tensor(melody_feature).float().to(device)


Model 1: 

Generated Lyrics: hiya we sleep would the saw someone and were down lovers stoned em and their saw that it now that hell room dead sleep 
 the day to live deceiving those loved take it down pain for me would fly ends brought enough easy 
 oh face my bell never smile the rain room of the at hell show goodbye to take it incomprehensible face by room show looks live mad is enough many times deep dead of room room down in dry sleep 
 let me their and gray she says your door ho late it sleep smile you were free low and gimme sleep 
 he see 
 i could marry 
 watching go 
 ah a black big god breaks their care as big glimpse for my knees me 
 someone gotta mess in their arms up to no questions ho na na screwing out black 
 uh ho 
 of its getting thinkin room bitch like i saw the lonely we would it could marry now 
 mess wrong 
 room and out the door late all their lonely in their lonely room enough could just melodies 
 and a glimpse ya stapled and gone some face in time bitch whats

/tmp/ipykernel_1783929/701868571.py:34: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  melody_feature_tensor = torch.tensor(melody_feature).float().to(device)


Model 2: 

Generated Lyrics: hiya trunk and it is to let you hes 
 imaginary let those beat me off with you really think i had not free or sad baby ever are the drapes just whoa that would it with me or go down while your got one got them 
 baby without you everything its so time right on yourself repeat 
 come someone else we say the oldies she too much i chaka and dust you should never wrong 
 let a fries cuz im got me down for a body 
 seduce i told the you got a dimension i cant give me goodbye sick you want it is gettin whoa gonna take a babies say for me why 
 she caught it right on my song 
 open anymore oh 
 beware 
 survives 
 run here must grow out on some no piece 
 cause they said impossible for 
 let this right again some my confirmation got me 
 stay street to them in london weary yeah that smokin i was and tell me honey love honey is some kind your my heart kiss my heartaches whos 
 in here is it just want me felt 
 done i get police 
 just call you leave my heavenly wou

## Word - barbie

In [ ]:
# Generate lyrics for a specific song
initial_word = 'barbie'
song_index = 3
chosen_model = [model1, model2]
print(f"Song {song_index + 1}: {initial_word}\n")

for model in chosen_model:
  generated_lyrics = generate_text(model, X_test_load[song_index], word_model, initial_word, song_index)
  # print model
  print(f"Model {chosen_model.index(model) + 1}: \n")
  generated_lyrics_with_newline = generated_lyrics
  generated_lyrics_with_newline = generated_lyrics_with_newline.replace('newline', '\n')
  print(f"Generated Lyrics: {generated_lyrics_with_newline}\n")
     # Evaluate lyrics and store results
  sim_1gram, sim_2gram, sim_3gram, sim_4gram = evaluate_lyrics(
      generated_lyrics, song_index
  )

  # Append results to the corresponding lists
  if model == model1:  # Model 1
      results_model1["1gram"].append(sim_1gram)
      results_model1["2gram"].append(sim_2gram)
      results_model1["3gram"].append(sim_3gram)
      results_model1["4gram"].append(sim_4gram)
  else:  # Model 2
      results_model2["1gram"].append(sim_1gram)
      results_model2["2gram"].append(sim_2gram)
      results_model2["3gram"].append(sim_3gram)
      results_model2["4gram"].append(sim_4gram)

  print(f"1-Gram Cosine Similarity: {sim_1gram:.4f}")
  print(f"2-Gram Cosine Similarity: {sim_2gram:.4f}")
  print(f"3-Gram Cosine Similarity: {sim_3gram:.4f}")
  print(f"4-Gram Cosine Similarity: {sim_4gram:.4f}")


Song 4: barbie



/tmp/ipykernel_1783929/701868571.py:34: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  melody_feature_tensor = torch.tensor(melody_feature).float().to(device)


Model 1: 

Generated Lyrics: barbie 
 that wants my eyes deceiving door up which door glass dead 
 never said your mind near the eye with a hole day 
 which door 
 never lie im a getting pity show up 
 room and your love out in lovers up watching no depths woman in the depths and their eyes for youd because bad open together bitch you both closer for it leave some late late is forever both for im ho slit bitch the monkey the late 
 stop away 
 deceiving of deceiving dead turning high getting into the bottle of these thinking 
 beautiful american 
 she said 
 is coming her great which wild which door room dead deceiving watching ya coming out 
 i would never done you live bitch like you hell dead smile 
 singin picking which bottle for him were had know why can i incomprehensible 
 id me were 
 and your mind but i 
 wrapped dead hungry im a hole oh with black late to reach myself meet sing i see they must said ya i said 
 just woman 
 to myself room away home 
 fly down in vain im not l

/tmp/ipykernel_1783929/701868571.py:34: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  melody_feature_tensor = torch.tensor(melody_feature).float().to(device)


Model 2: 

Generated Lyrics: barbie cause he did is real rock like through the light of the poison 
 ive know 
 boy i bit 
 when i fabulous call it ill almost brought you goes on things for a chance did make a things on up for all the city of fact you want it out of the name is faith 
 listen is when i seem to shoot lets check too right again up the beware 
 we swore it up runnin when 
 baby impossible him if you gonna live them everything and he all about the hundred heaven 
 and forwards a brains if i heard that there is the whole word from angles too 
 and im wondering 
 ocean yeah and looking to the bone in the funk 
 yes here 
 im pills 
 he had a hands 
 intergalactic like a puppy that they play we just lie 
 ill stop 
 got somebody can let her tryin you need it in cheeks 
 got tha gimme shit uh yeah a showgirl ill sleep back at promise 
 im a girl 
 abacab to kiss another race at his wedding your way back of old excuse 
 so much 
 why the game alone 
 ah life 
 da da da ba daa o

## Word - hi

In [ ]:
# Generate lyrics for a specific song
initial_word = 'hi'
song_index = 3
chosen_model = [model1, model2]
print(f"Song {song_index + 1}: {initial_word}\n")

for model in chosen_model:
  generated_lyrics = generate_text(model, X_test_load[song_index], word_model, initial_word, song_index)
  # print model
  print(f"Model {chosen_model.index(model) + 1}: \n")
  generated_lyrics_with_newline = generated_lyrics
  generated_lyrics_with_newline = generated_lyrics_with_newline.replace('newline', '\n')
  print(f"Generated Lyrics: {generated_lyrics_with_newline}\n")
    # Evaluate lyrics and store results
  sim_1gram, sim_2gram, sim_3gram, sim_4gram = evaluate_lyrics(
      generated_lyrics, song_index
  )

  # Append results to the corresponding lists
  if model == model1:  # Model 1
      results_model1["1gram"].append(sim_1gram)
      results_model1["2gram"].append(sim_2gram)
      results_model1["3gram"].append(sim_3gram)
      results_model1["4gram"].append(sim_4gram)
  else:  # Model 2
      results_model2["1gram"].append(sim_1gram)
      results_model2["2gram"].append(sim_2gram)
      results_model2["3gram"].append(sim_3gram)
      results_model2["4gram"].append(sim_4gram)

  print(f"1-Gram Cosine Similarity: {sim_1gram:.4f}")
  print(f"2-Gram Cosine Similarity: {sim_2gram:.4f}")
  print(f"3-Gram Cosine Similarity: {sim_3gram:.4f}")
  print(f"4-Gram Cosine Similarity: {sim_4gram:.4f}")


Song 4: hi



/tmp/ipykernel_1783929/701868571.py:34: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  melody_feature_tensor = torch.tensor(melody_feature).float().to(device)


Model 1: 

Generated Lyrics: hi in god 
 no hungry move into my finger watching me room down late like it in their side i fly giddy letter so it slowly 
 half bitch no hole and green of your my view 
 no one take me meet friends up me 
 never hole 
 and while now will bring getting wild go show me like my saw it show i all tomorrows 
 the reason 
 that god 
 i just why saw someone won on the crowd room 
 take their saw my rise dead in that hell 
 cause at full one dead dead in my that ever knew to some black which home baby eyes for me so noise dead god 
 down in their getting to sing without beat bitch im coming dont 
 oh and singing dead would up with their when i one been the saw it in world for the a friend 
 lookin forget 
 bitch and before i sleep 
 closer room smile rain day desire to shine down and god into the poison 
 bring their best down and just said it is lonely dead crowd in a depths room 
 so watching their stab out in the depths in vain 
 with ten door through my big s

/tmp/ipykernel_1783929/701868571.py:34: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  melody_feature_tensor = torch.tensor(melody_feature).float().to(device)


Model 2: 

Generated Lyrics: hi gonna you walk around was a heartache 
 to be what it home back 
 and get laughs 
 everybody fair 
 words could they say you got you to make loving make love me 
 romance that you must make you 
 candle chic 
 to party 
 barry is so cold of his kiss that defense monday let all hell 
 and beat him outside effect too strong dream can be out alright 
 in year aint 
 if i really wanna talk 
 ring 
 forgive what but i am the skies 
 please is it side 
 i cant yo yes i dont look at the point 
 sing and interesting to tumble 
 had to keep ye oh 
 whats place your jive listen of egyptian tide 
 like tell my block 
 right here his body 
 gon 
 but you to have long enough on a raised in her shady escape over my first time after a natural 
 at rest will be another place to yeah 
 if i have your eyes 
 talk id flava ammo believe 
 lie 
 anyway i my mind 
 off of my babys all the spin he was is id see the day not a place 
 bop 
 im a beats and sexy dont come on it be

# **Test Song 5**

## Word - all

In [ ]:
# Generate lyrics for a specific song
initial_word = 'all'
song_index = 4
chosen_model = [model1, model2]
print(f"Song {song_index + 1}: {initial_word}\n")

for model in chosen_model:
  generated_lyrics = generate_text(model, X_test_load[song_index], word_model, initial_word, song_index)
  # print model
  print(f"Model {chosen_model.index(model) + 1}: \n")
  generated_lyrics_with_newline = generated_lyrics
  generated_lyrics_with_newline = generated_lyrics_with_newline.replace('newline', '\n')
  print(f"Generated Lyrics: {generated_lyrics_with_newline}\n")
     # Evaluate lyrics and store results
  sim_1gram, sim_2gram, sim_3gram, sim_4gram = evaluate_lyrics(
      generated_lyrics, song_index
  )

  # Append results to the corresponding lists
  if model == model1:  # Model 1
      results_model1["1gram"].append(sim_1gram)
      results_model1["2gram"].append(sim_2gram)
      results_model1["3gram"].append(sim_3gram)
      results_model1["4gram"].append(sim_4gram)
  else:  # Model 2
      results_model2["1gram"].append(sim_1gram)
      results_model2["2gram"].append(sim_2gram)
      results_model2["3gram"].append(sim_3gram)
      results_model2["4gram"].append(sim_4gram)


  print(f"1-Gram Cosine Similarity: {sim_1gram:.4f}")
  print(f"2-Gram Cosine Similarity: {sim_2gram:.4f}")
  print(f"3-Gram Cosine Similarity: {sim_3gram:.4f}")
  print(f"4-Gram Cosine Similarity: {sim_4gram:.4f}")


Song 5: all

Model 1: 

Generated Lyrics: all dead wail 
 because their room 
 their two one hell see we would take me 
 theres move into their door naturally its incomprehensible their yo marry us we must 
 cause gimme the street through my reason for now oh am i knew go just a hole wants me sleep 
 up 
 both to live this never down closer by wrong 
 sleep things id closer glass 
 so bedouin no reason rock out dead 
 ready wont stab get it easy shelter at my sleep at a white its not 
 oh oh oh oh hey in which two lonely dead black bitch and their mind through than words 
 had so free 
 honey god when forget my dead twist 
 and baby right 
 god once it easy dead say want 
 ah ho lives your reason the facing their door show for a pity ya coming into the black it easy words and remember as late of 
 we are yours guitar to try to that would up dead forgotten 
 so to marry 
 fly their gotta understand in a bottle 
 id never least christ 
 my place into god mend

Real Lyrics: all the small 

/tmp/ipykernel_1783929/701868571.py:34: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  melody_feature_tensor = torch.tensor(melody_feature).float().to(device)
/tmp/ipykernel_1783929/701868571.py:34: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  melody_feature_tensor = torch.tensor(melody_feature).float().to(device)


Model 2: 

Generated Lyrics: all the peace else be filled away 
 searching 
 and would love you too much one question with one just apart take it 
 model when up 
 let me high like it roll 
 leave with down from tomorrows 
 i said 
 and you try again 
 i die hope neither than you 
 feel it all the started walk eyes from it 
 and is on my faith comes and visit the whole secret were around you were feeling time in which im goodbye by box baby 
 baby matter it 
 when stiff live everything down and angles now than all it with you with you are no dreams 
 so when shes more i need most scandals 
 upside the have a do all in every way for back 
 cause that we last day when they gotta be new life 
 and remember that him on my whole morning together peace side be love 
 
 but it 
 come out 
 well live my package not one day 
 and ill wear me in the border is not only one lizards with a floor 
 baby has to say 
 by touch by

Real Lyrics: all the small things 
 true care truth brings 
 ill take o

## Word - the

In [ ]:
# Generate lyrics for a specific song
initial_word = 'the'
song_index = 4
chosen_model = [model1, model2]
print(f"Song {song_index + 1}: {initial_word}\n")

for model in chosen_model:
  generated_lyrics = generate_text(model, X_test_load[song_index], word_model, initial_word, song_index)
  # print model
  print(f"Model {chosen_model.index(model) + 1}: \n")
  generated_lyrics_with_newline = generated_lyrics
  generated_lyrics_with_newline = generated_lyrics_with_newline.replace('newline', '\n')
  print(f"Generated Lyrics: {generated_lyrics_with_newline}\n")
     # Evaluate lyrics and store results
  sim_1gram, sim_2gram, sim_3gram, sim_4gram = evaluate_lyrics(
      generated_lyrics, song_index
  )

  # Append results to the corresponding lists
  if model == model1:  # Model 1
      results_model1["1gram"].append(sim_1gram)
      results_model1["2gram"].append(sim_2gram)
      results_model1["3gram"].append(sim_3gram)
      results_model1["4gram"].append(sim_4gram)
  else:  # Model 2
      results_model2["1gram"].append(sim_1gram)
      results_model2["2gram"].append(sim_2gram)
      results_model2["3gram"].append(sim_3gram)
      results_model2["4gram"].append(sim_4gram)

  print(f"1-Gram Cosine Similarity: {sim_1gram:.4f}")
  print(f"2-Gram Cosine Similarity: {sim_2gram:.4f}")
  print(f"3-Gram Cosine Similarity: {sim_3gram:.4f}")
  print(f"4-Gram Cosine Similarity: {sim_4gram:.4f}")


Song 5: the

Model 1: 

Generated Lyrics: the everything id know love 
 and thinking me lonely dead lonely crowd upside me 
 full to me one saw my black which stab and had gimme so easy wrapped dead gotta marry in hell em 
 turning down til it up black up me room would 
 both my its your arms of my arms watching it down of which hungry youd watching 
 im so live in many times just 
 sleep mess 
 gimme the depths 
 with me for so it down of place and now thinkin mend nowhere could be getting their two of the attitude upside my have no that big pain dead 
 photograph for servant and their lonely down our pity 
 id 
 cause fly near 
 venus a friend get up watching dead more live their big as bedouin a car what would relate together take so done help both of mind 
 catch dry which shes to take her moon da had a wild her would still would give peace part 
 closer far by of now god brought 
 shoe strong ringing attitude perfect yours i swear is coming i dont by black im in 
 i take

Real Lyr

/tmp/ipykernel_1783929/701868571.py:34: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  melody_feature_tensor = torch.tensor(melody_feature).float().to(device)
/tmp/ipykernel_1783929/701868571.py:34: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  melody_feature_tensor = torch.tensor(melody_feature).float().to(device)


Model 2: 

Generated Lyrics: the boardwalk such out 
 so rain has not close to be opened sense to be on tie just time i dont buy it 
 da la la la la be a posters 
 when only love is silken rock your and ad 
 cause i do like to shoot into the same lonely 
 you do you and such a reason 
 still know you tried 
 but it it aint i took ten light 
 plan 
 aint aint were at treat you must be never time alone again 
 true weighing easy or you may change for mo di ya 
 and hope one girl im baby up from a song thats no give are 
 do you were serious good to pay me such that i cant let that i wants to bite your reflection my secrets guitar 
 alleluia and fill a get there to love again im decision to lots queen making you 
 ill break rooms 
 save it for you teach me the dark city of these make me 
 da the one day cause where me for two away 
 to make me you work i love 
 oh or and telling

Real Lyrics: all the small things 
 true care truth brings 
 ill take one lift 
 your ride best trip 
 always 

## Word - small

In [ ]:
# Generate lyrics for a specific song
initial_word = 'small'
song_index = 4
chosen_model = [model1, model2]
print(f"Song {song_index + 1}: {initial_word}\n")

for model in chosen_model:
  generated_lyrics = generate_text(model, X_test_load[song_index], word_model, initial_word, song_index)
  # print model
  print(f"Model {chosen_model.index(model) + 1}: \n")
  generated_lyrics_with_newline = generated_lyrics
  generated_lyrics_with_newline = generated_lyrics_with_newline.replace('newline', '\n')
  print(f"Generated Lyrics: {generated_lyrics_with_newline}\n")
     # Evaluate lyrics and store results
  sim_1gram, sim_2gram, sim_3gram, sim_4gram = evaluate_lyrics(
      generated_lyrics, song_index
  )

  # Append results to the corresponding lists
  if model == model1:  # Model 1
      results_model1["1gram"].append(sim_1gram)
      results_model1["2gram"].append(sim_2gram)
      results_model1["3gram"].append(sim_3gram)
      results_model1["4gram"].append(sim_4gram)
  else:  # Model 2
      results_model2["1gram"].append(sim_1gram)
      results_model2["2gram"].append(sim_2gram)
      results_model2["3gram"].append(sim_3gram)
      results_model2["4gram"].append(sim_4gram)


  print(f"1-Gram Cosine Similarity: {sim_1gram:.4f}")
  print(f"2-Gram Cosine Similarity: {sim_2gram:.4f}")
  print(f"3-Gram Cosine Similarity: {sim_3gram:.4f}")
  print(f"4-Gram Cosine Similarity: {sim_4gram:.4f}")


Song 5: small

Model 1: 

Generated Lyrics: small bitch of anywhere 
 because many enough depths and he bitch about in god on baby 
 ah never said about and facing her dreams incomprehensible try out their nigga we embarrassment my care that you sleep near 
 lonely for underground deceiving my depths their for my pride in the glimpse brought door back must take ya coming 
 maybe for big free black eye 
 all their words who easy 
 lets myself i remember into attitude no hole 
 im coming full arms feeling id a car 
 i want you to take 
 he never knew 
 photograph as sweet because their saw me through their part of their life late to a place worth 
 take it fly die sleep baby lean door 
 getting brought room attitude day in 
 one 
 throw eternally brother 
 uh she wants your arms lonely of hell smile late melodies my part dead of their mind and lonely 
 make another rain 
 let me 
 goodbye im lost their reason tonight had 
 fly down for i swear god 
 id pity 
 more that sleep lonely lonel

/tmp/ipykernel_1783929/701868571.py:34: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  melody_feature_tensor = torch.tensor(melody_feature).float().to(device)
/tmp/ipykernel_1783929/701868571.py:34: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  melody_feature_tensor = torch.tensor(melody_feature).float().to(device)


Model 2: 

Generated Lyrics: small these before as never need you can build you can think 
 there the crowd steel shine knows better listen me let thither in line 
 and the light 
 and i stay through 
 you were 
 a time 
 you seeing sings 
 i gotta say without each days put me do any world 
 that we should thoughts 
 help me out where looks can people dont you find for you mean that by she was sorry gotta turn to mess 
 keeping view fills 
 now 
 ill have so yes oh oh 
 i wonder 
 and wondering shes take up and fight nights and they sign today 
 and i be 
 lets try you need is the past die and comin and your fuckin romance sails for the street be your father 
 freedom into keeping free 
 i bought you gettin to me from 
 were in a step your clothes 
 cant secrets must taking 
 up how to you feel any time mmm 
 lonely 
 seasons anyway we wish run like me with being 
 care 
 i know 
 they need been too in the words in life

Real Lyrics: all the small things 
 true care truth brings 
 ill 

# **results**

In [ ]:
# Calculate mean results for each model
def calculate_mean_results(results):
    return {key: np.mean(values) if values else 0 for key, values in results.items()}

mean_results_model1 = calculate_mean_results(results_model1)
mean_results_model2 = calculate_mean_results(results_model2)

# Print summary
print("Summary of Results:")
print(f"Model 1 Mean Results: {mean_results_model1}")
print(f"Model 2 Mean Results: {mean_results_model2}")

Summary of Results:
Model 1 Mean Results: {'1gram': 0.1826923578698188, '2gram': 0.22032057583332063, '3gram': 0.277560206707567, '4gram': 0.37563665717840194}
Model 2 Mean Results: {'1gram': 0.39602609417800394, '2gram': 0.3025585362421615, '3gram': 0.3084555741931711, '4gram': 0.3476228304207325}
